# Relatórios de Tráfego com Custo

### Bibliotecas Utilizadas

In [1]:
import pandas
import numpy
import pyarrow
import os
import pathlib
import teradatasql
import warnings

# import openpyxl
# from openpyxl.utils.dataframe import dataframe_to_rows
from dotenv import load_dotenv

### Environment Variables

In [2]:
MES_TAP = "202511"
MES_TARIFA = "202511"

load_dotenv()
BASE_TARIFA = (
    str(os.environ.get("BASE_TARIFA"))
    + f"/{MES_TARIFA[0:4]}"
    + f"/{MES_TARIFA[0:4]}.{MES_TARIFA[4:6]}_Custo.xlsx"
)
BASE_TAP = pathlib.Path(
    str(os.environ.get("BASE_TAP")) + f"/NACIONAL/{MES_TAP}"  # f"/{MES_TAP}"
)  # f"/NACIONAL/{MES_TAP}") | f"/{MES_TAP}"
BASE_SINALIZACAO = pathlib.Path(str(os.environ.get("BASE_SINALIZACAO")) + f"/{MES_TAP}")

ANALISE_DIARIAS_B2C = str(os.environ.get("ANALISE_DIARIAS_B2C"))
ANALISE_DIARIAS_B2B = str(os.environ.get("ANALISE_DIARIAS_B2B"))

BASE_FINAL = str(os.environ.get("BASE_FINAL")) + f"/{MES_TAP}"
REPORTS_FOLDER = str(os.environ.get("REPORTS")) + f"/{MES_TAP}"

FUTURE_WARNINGS = str(os.environ.get("FUTURE_WARNINGS"))

TERADATA_HOST = str(os.environ.get("TERADATA_HOST"))
TERADATA_USER = str(os.environ.get("TERADATA_USER"))
TERADATA_PASSWORD = str(os.environ.get("TERADATA_PASSWORD"))

# Supress Future Warnings
warnings.simplefilter(action=FUTURE_WARNINGS, category=FutureWarning)
pandas.options.mode.copy_on_write = True

### Carregamento da base de tarifas

In [ ]:
tarifas = pandas.read_excel(BASE_TARIFA, sheet_name="TGR", skiprows=1)

tarifas.rename(
    columns={
        "Code": "PMN (Settlement) TADIG Code",
        "DATA": "Tarifa_Dados",
        "SMS MO": "Tarifa_SMS",
        "Voice MO": "Tarifa_MOC",
        "Voice MT": "Tarifa_MTC",
    },
    inplace=True,
)
tarifas.drop_duplicates(subset="PMN (Settlement) TADIG Code", keep="last", inplace=True)
tarifas[
    [
        "PMN (Settlement) TADIG Code",
        "Tarifa_Dados",
        "Tarifa_MOC",
        "Tarifa_MTC",
        "Tarifa_SMS",
    ]
]

cols = ["Tarifa_Dados", "Tarifa_MOC", "Tarifa_MTC", "Tarifa_SMS"]
for col in cols:
    tarifas[tarifas[col] == "Gross"] = 0.0
    tarifas[col] = tarifas[col].astype("float")

tarifas.drop(
    columns=[
        "Deal Status",
        "CHECK PLMN",
        "Unnamed: 19",
        "Unnamed: 20",
        "Unnamed: 21",
        "Unnamed: 22",
        "Sponsor",
        "Concept",
        "Direction",
        "GROUP NAME",
        "Unnamed: 5",
        "Operator",
        "Country",
        "Region",
        "Unnamed: 9",
        "Start",
        "Final",
        "Deal Currency",
    ],
    inplace=True,
)
tarifas.info()

### Carregamento de Vários Arquivos TAP/Sinalizacao e Junção dos mesmos

#### Carregamento TAP

In [3]:
paths = sorted(BASE_TAP.iterdir(), key=os.path.getmtime)
bases = []
# print(paths)

for file in paths:
    if (
        str(file).endswith("final.csv")
        or str(file).endswith(".parquet")
        or str(file).endswith(".zip")
        or str(file).endswith(".csv.gz")
    ):
        continue
    try:
        print(f"Loading file: {file}")
        bases.append(
            pandas.read_csv(
                file,
                sep=";",
                decimal=",",
                encoding="utf_16",
                dtype={
                    "IMSI": str,
                    "MSISDN": str,
                    "Settlement Gross Charge - TAP Local Currency": str,
                    "Number of CDRs": str,
                    "Charged Events": str,
                    "Charged Minutes": str,
                    "Charged MB": str,
                },
                low_memory=False,
                skiprows=0,
            )
        )
    except Exception:
        ...

tap = pandas.concat(bases)
del bases

tap["MSISDN_fix"] = tap["MSISDN"].str.zfill(20)
tap["MSISDN_fix"] = tap["MSISDN_fix"].str.slice(9)

# tap.to_csv(str(BASE_TAP) + "/final.csv.gz", sep=";", decimal=",", encoding="utf_16")

for col in [
    "PMN (Settlement) Country",
    "PMN (Settlement) TADIG Code",
    "Year-Month (Settlement)",
    "Call Type",
    "TAP File (Current) Name",
    "IMSI",
    "MSISDN",
    "Device TAC",
]:
    try:
        tap[col] = tap[col].astype("category")
    except:
        pass

try:
    tap["TAP File (Current) Processing Date"] = pandas.to_datetime(
        tap["TAP File (Current) Processing Date"], dayfirst=True
    )
except KeyError:
    ...

try:
    tap["Date (Call)"] = pandas.to_datetime(tap["Date (Call)"], dayfirst=True)
except KeyError:
    ...

for col in [
    "Number of CDRs",
    "Charged Events",
    "Charged Minutes",
    "Charged MB",
    "Settlement Gross Charge - TAP Local Currency",
    "Settlement Charge - USD",
    "Settlement Tax - USD",
    "Settlement Gross Charge - USD",
    "Settlement Gross Charge - EUR",
]:
    try:
        tap[col] = tap[col].astype(str).str.replace(".", "")
        tap[col] = tap[col].astype(str).str.replace(",", ".")
        tap[col] = tap[col].astype(str).str.replace("nan", "")
        tap[col] = pandas.to_numeric(tap[col], downcast="float")
    except KeyError:
        continue

# tap["MSISDN_fix"] = tap["MSISDN_fix"].astype(str).str.replace(".0", "")
# tap["MSISDN_fix"] = tap["MSISDN_fix"].str.replace("<NA>", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].astype("category")

# tap["IMSI"] = tap["IMSI"].astype(str).str.replace(".0", "")
# tap["IMSI"] = tap["IMSI"].str.replace("<NA>", "")
tap["IMSI"] = tap["IMSI"].astype("category")

try:
    tap.drop(columns=["Unnamed: 0"], inplace=True)
except KeyError:
    pass
tap["Charged MB"].sum() / (1024**2)

Loading file: C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\NACIONAL\202511\03. ROAMING NACIONAL 30K - OUTBOUND – Custo Mensal Segmentado CdG.csv
Loading file: C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\NACIONAL\202511\03. ROAMING NACIONAL 30K - OUTBOUND – Custo Mensal Segmentado CdG (1).csv
Loading file: C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\NACIONAL\202511\03. ROAMING NACIONAL 30K - OUTBOUND – Custo Mensal Segmentado CdG (2).csv
Loading file: C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\NACIONAL\202511\03. ROAMING NACIONAL 30K - OUTBOUND – Custo Mensal Segmentado CdG (3).csv
Loading file: C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional

np.float64(127.03654169654845)

#### Carregamento Sinalização

In [ ]:
def load_signaling_files():
    try:
        paths = sorted(BASE_SINALIZACAO.iterdir(), key=os.path.getmtime)

        bases = []
        for file in paths:
            if str(file).endswith("final.csv.gz") or str(file).endswith(".parquet"):
                continue
            print(str(file))
            bases.append(
                pandas.read_csv(
                    file,
                    sep=",",
                    decimal=".",
                    encoding="utf_8-sig",
                    dtype={"IMSI": str, "MSISDN": str},
                    low_memory=False,
                )
            )
        sinalizacao = pandas.concat(bases)
        del bases

        sinalizacao["IMSI"] = pandas.to_numeric(sinalizacao["IMSI"], downcast="integer")
        sinalizacao["MSISDN"] = pandas.to_numeric(
            sinalizacao["MSISDN"], downcast="integer", errors="coerce"
        )
        sinalizacao["_MSISDN"] = pandas.to_numeric(
            sinalizacao["_MSISDN"], downcast="integer", errors="coerce"
        )
        sinalizacao["Calling Party IMSI"] = pandas.to_numeric(
            sinalizacao["Calling Party IMSI"], downcast="integer"
        )

        import numpy

        # Mesclando colunas MSISDN e _MSISDN
        conditions = [sinalizacao["MSISDN"].notna(), sinalizacao["_MSISDN"].notna()]
        values = [sinalizacao["MSISDN"], sinalizacao["_MSISDN"]]
        sinalizacao["MSISDN_fix"] = numpy.select(conditions, values, default=pandas.NA)

        # Otimizando as colunas para não estourar memória
        sinalizacao["MSISDN_fix"] = (
            sinalizacao["MSISDN_fix"].astype(str).str.replace(".0", "")
        )
        sinalizacao["MSISDN_fix"] = sinalizacao["MSISDN_fix"].str.replace("<NA>", "")
        sinalizacao["MSISDN_fix"] = pandas.to_numeric(
            sinalizacao["MSISDN_fix"], downcast="integer"
        )
        sinalizacao["MSISDN_fix"] = sinalizacao["MSISDN_fix"] - 5500000000000

        conditions = [
            sinalizacao["IMSI"].notna(),
            sinalizacao["Calling Party IMSI"].notna(),
        ]
        values = [sinalizacao["IMSI"], sinalizacao["Calling Party IMSI"]]
        sinalizacao["IMSI_fix"] = numpy.select(conditions, values, default=pandas.NA)

        conditions = [
            sinalizacao["Originating Operator Country Name"].notna(),
            sinalizacao["Network Operator Country Name"].notna(),
        ]
        values = [
            sinalizacao["Originating Operator Country Name"],
            sinalizacao["Network Operator Country Name"],
        ]
        sinalizacao["Country"] = numpy.select(conditions, values, default=pandas.NA)

        sinalizacao.rename(columns={"01 Date": "Date (Call) YYYYMMDD"}, inplace=True)
        sinalizacao.drop(
            columns=[
                "IMSI",
                "Calling Party IMSI",
                "MSISDN",
                "_MSISDN",
                "Originating Operator Country Name",
                "Network Operator Country Name",
            ],
            inplace=True,
        )

        sinalizacao["IMSI_fix"] = (
            sinalizacao["IMSI_fix"].astype(str).str.replace(".0", "")
        )
        sinalizacao["IMSI_fix"] = sinalizacao["IMSI_fix"].str.replace("<NA>", "")
        sinalizacao["IMSI_fix"] = sinalizacao["IMSI_fix"].astype("category")
        sinalizacao["Country"] = sinalizacao["Country"].astype("category")
        sinalizacao["MSISDN_fix"] = sinalizacao["MSISDN_fix"].astype("category")
        sinalizacao["Date (Call) YYYYMMDD"] = pandas.to_datetime(
            sinalizacao["Date (Call) YYYYMMDD"], yearfirst=True
        )

        sinalizacao.rename(columns={"IMSI_fix": "IMSI"}, inplace=True)

        sinalizacao[sinalizacao["Country"] != "Brazil"].to_parquet(
            str(BASE_SINALIZACAO) + "/final.parquet"
        )
        sinalizacao[sinalizacao["Country"] == "Brazil"].to_parquet(
            str(BASE_SINALIZACAO) + "/final_brazil.parquet"
        )

        sinalizacao.info()
    except Exception as e:
        print(e)
        return None


load_signaling_files()

C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\Sinalizacao\202412\IMSI_List_SCCP_Diameter_LU_OK_Day_Before_SCCP_2024-12-01.csv
C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\Sinalizacao\202412\IMSI_List_SCCP_Diameter_LU_OK_Day_Before_Diameter_2024-12-01.csv
C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\Sinalizacao\202412\IMSI_List_SCCP_Diameter_LU_OK_Day_Before_SCCP_2024-12-02.csv
C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\Sinalizacao\202412\IMSI_List_SCCP_Diameter_LU_OK_Day_Before_Diameter_2024-12-02.csv
C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\Sinalizacao\202412\IMSI_List_SCCP_Diameter_LU_OK_Day_Before_SCCP_2024-12-03.csv
C:\Users\a0153041\OneD

### Segmentação

#### Segmentação por Sinalização

In [ ]:
# Exportar informações da Sinalização para segmentação

SEGMENT_FOLDER = f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/01-LINHAS_PARA_SEGMENTAR/{MES_TAP}/"
COLS_TO_EXPORT = ["IMSI", "MSISDN_fix"]

# linhas = tap[
#            tap['MSISDN_fix'].astype(str).str.contains('nan')!=True
#        ].groupby(by=['IMSI', 'MSISDN_fix'], observed=True).agg({
#            "Settlement Gross Charge - USD":"sum"
#        }).reset_index()

# Carregar informações de sinalização
try:
    # Exportar informações para segmentação
    linhas_sinalizacao = (
        sinalizacao[sinalizacao["MSISDN_fix"].astype(str).str.contains("nan") != True]
        .groupby(by=["IMSI", "MSISDN_fix"], observed=True)
        .agg({"Country": "count"})
        .reset_index()
    )
except NameError:
    sinalizacao = pandas.read_parquet(str(BASE_SINALIZACAO) + "/final.parquet")

    linhas_sinalizacao = (
        sinalizacao[sinalizacao["MSISDN_fix"].astype(str).str.contains("nan") != True]
        .groupby(by=["IMSI", "MSISDN_fix"], observed=True)
        .agg({"Country": "count"})
        .reset_index()
    )
finally:
    sinalizacao["MSISDN_fix"] = (
        sinalizacao["MSISDN_fix"].astype(str).str.replace(".0", "")
    )
    sinalizacao["MSISDN_fix"] = sinalizacao["MSISDN_fix"].str.replace("<NA>", "")
    sinalizacao["MSISDN_fix"] = sinalizacao["MSISDN_fix"].astype("category")

    # pandas.concat(
    #    [linhas[COLS_TO_EXPORT], linhas_sinalizacao[COLS_TO_EXPORT]]
    # )
    linhas_sinalizacao[COLS_TO_EXPORT].drop_duplicates(
        subset=["IMSI", "MSISDN_fix"], keep="last"
    ).to_csv(
        SEGMENT_FOLDER + "linhas_para_segmentar.csv",
        sep=",",
        decimal=".",
        encoding="utf_8",
        header=False,
        index=False,
        mode="w",
    )

    for col in sinalizacao:
        if col != "Date (Call) YYYYMMDD" and col != "IMSI" and col != "MSISDN_fix":
            sinalizacao[col] = sinalizacao[col].astype("category")

    sinalizacao["MSISDN_fix"].unique().tolist()

#### Segmentação por TAP

In [4]:
# Exportar informações do TAP para segmentação

# tap = pandas.read_parquet(
#    r'C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\202409\202409_fixed.parquet')

COLS_TO_EXPORT = ["IMSI", "MSISDN_fix"]
try:
    temp = tap[["IMSI", "MSISDN_fix", "Settlement Gross Charge - USD"]]
    temp["IMSI"] = temp["IMSI"].astype(str)
    temp["MSISDN_fix"] = temp["MSISDN_fix"].astype(str)

    temp = temp[temp["MSISDN_fix"] != "nan"]

    linhas = (
        temp.groupby(by=["IMSI", "MSISDN_fix"])
        .agg({"Settlement Gross Charge - USD": "sum"})
        .reset_index()
        .drop_duplicates(subset=["IMSI", "MSISDN_fix"], keep="last")
    )
except KeyError:
    temp = tap[["IMSI", "MSISDN_fix", "Settlement Gross Charge - TAP Local Currency"]]
    temp["IMSI"] = temp["IMSI"].astype(str)
    temp["MSISDN_fix"] = temp["MSISDN_fix"].astype(str)

    temp = temp[temp["MSISDN_fix"] != "nan"]

    linhas = (
        temp.groupby(by=["IMSI", "MSISDN_fix"])
        .agg({"Settlement Gross Charge - TAP Local Currency": "sum"})
        .reset_index()
        .drop_duplicates(subset=["IMSI", "MSISDN_fix"], keep="last")
    )

linhas = linhas[linhas["MSISDN_fix"] != "00000000000"]

try:
    linhas[COLS_TO_EXPORT].to_csv(
        f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/01-LINHAS_PARA_SEGMENTAR/{MES_TAP}/linhas_para_segmentar.csv",
        sep=",",
        decimal=".",
        encoding="utf_8",
        header=False,
        index=False,
        mode="w",
    )
except OSError:
    os.mkdir(
        f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/01-LINHAS_PARA_SEGMENTAR/{MES_TAP}"
    )
    linhas[COLS_TO_EXPORT].to_csv(
        f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/01-LINHAS_PARA_SEGMENTAR/{MES_TAP}/linhas_para_segmentar.csv",
        sep=",",
        decimal=".",
        encoding="utf_8",
        header=False,
        index=False,
        mode="w",
    )

##### VIP

In [3]:
import pandas

vip = pandas.read_excel(
    r"c:\Users\a0153041\Downloads\Relatório C Level (Aceleração Comercial)_20251006.xlsx",
    sheet_name="Relatório_C-level (Qualificado)",
    dtype={"Celular": str},
)
vip.head(5)

,Função comercial do contato da conta: ID,Conta: Segmentação do Cliente,Conta: Segmento valor,Conta: Código do Grupo,Conta: Código do cliente,Conta: CPF/CNPJ,Conta: Razão social,Contato,Celular,Email,...,NPS,COD_GRUPO,GRUPO_ECONOMICO,NOME_DIVISAO_EQUIPE,DIRETORIA_COMERCIAL,Parque Vivo?,Plano,Qtde. Vivo Travel,Vivo Travel Fat,Vivo Travel SVA
0,a0PHq00000ZPuPG,CORPORATE,Corporate V,4059104760,4059104760,59104760000191,TOYOTA DO BRASIL LTDA,JOAO CARLOS MACHADO PONTES,15998378377,jpontes@toyota.com.br,...,True,4.059105e+09,TOYOTA DO BRASIL LTDA,Edson Aparecido Ribeiro,CORPORATE ISE,1,SMART EMPRESAS 12GB TE,2.0,16.15,Vivo Travel Mensal Mundo Voz + Dados; Vivo Tra...
1,a0PHq00000ZWY61,TOP,TOP,4001276917,4086584901,86584901000193,UNIMED MACHADO COOPERATIVA DE TRABALHO MEDICO,CLAUDIO DE LIMA ALVES,35984286073,iara.miranda@unimedmachado.coop.br,...,True,4.001277e+09,UNIMED TRANSPORTE AEROMEDICA MG LTDA,Flavio Geraldo Aganete,TOP FSP,1,SMART EMPRESAS 12GB TE,2.0,18.31,Vivo Travel Mensal Europa Voz + Dados; Vivo Tr...
2,a0P1R00000VpqAP,TOP,TOP,4000907750,4000907750,907750000141,MINASBEB COMERCIO DE BEBIDAS LTDA,CRISTIANO HENRI DE SOUZA CUSTODIO DE SOUZA CUS...,35998252525,obz@minasbeb.com.br,...,True,4.000908e+09,MINASBEB COMERCIO DE BEBIDAS LTDA,Flavio Geraldo Aganete,TOP FSP,1,SMART EMPRESAS 12GB TE,2.0,12.98,Vivo Travel Mensal Europa Voz + Dados; Vivo Tr...
3,a0PHq00000YfROa,TOP,TOP,4048211585,4048211585,48211585000115,SOCIEDADE BRASILEIRA CAMINHO DE DAMASCO,MAX DE OLIVEIRA,.1150903030,max.oliveira@sbcdsaude.org.br,...,False,4.048212e+09,SOCIEDADE BRASILEIRA CAMINHO DE DAMASCO,Fabio Roberto Peres,TOP SP,0,NaN,NaN,NaN,NaN
4,a0PHq00000ZVxIc,TOP,TOP,4003224265,4003224265,3224265000161,LOGISTOCK LOGISTICA E SERVICOS LTDA,Valdirene Logistock,00000000000,financeiro@logistock.log.br,...,False,4.003224e+09,LOGISTOCK LOGISTICA E SERVICOS LTDA,0,TOP FSP,0,NaN,NaN,NaN,NaN


In [4]:
linhas = vip[["Conta: Código do cliente", "Celular"]]
linhas.head(5)

,Conta: Código do cliente,Celular
0,4059104760,15998378377
1,4086584901,35984286073
2,4000907750,35998252525
3,4048211585,.1150903030
4,4003224265,00000000000


In [5]:
linhas["Celular_fix"] = linhas["Celular"].str.lstrip("0")
linhas["Celular_fix"] = linhas["Celular_fix"].str.replace(".", "")
linhas

C:\Users\a0153041\AppData\Local\Temp\ipykernel_69528\3671751863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linhas["Celular_fix"] = linhas["Celular"].str.lstrip("0")
C:\Users\a0153041\AppData\Local\Temp\ipykernel_69528\3671751863.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linhas["Celular_fix"] = linhas["Celular_fix"].str.replace(".", "")


,Conta: Código do cliente,Celular,Celular_fix
0,4059104760,15998378377,15998378377
1,4086584901,35984286073,35984286073
2,4000907750,35998252525,35998252525
3,4048211585,.1150903030,1150903030
4,4003224265,00000000000,
...,...,...,...
23480,4061585865,99999999999,99999999999
23481,4062895792,99999999999,99999999999
23482,4067616128,99999999999,99999999999
23483,4069286763,99999999999,99999999999


In [6]:
REGEX_MSISDN = "[1-9]{2}9[0-9]{8}$"
linhas["VALID_MSISDN"] = (
    linhas["Celular_fix"]
    .astype(str)
    .str.replace(".0", "")
    .str.match(REGEX_MSISDN, na=False)
)
linhas

C:\Users\a0153041\AppData\Local\Temp\ipykernel_69528\1693295177.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linhas["VALID_MSISDN"] = (


,Conta: Código do cliente,Celular,Celular_fix,VALID_MSISDN
0,4059104760,15998378377,15998378377,True
1,4086584901,35984286073,35984286073,True
2,4000907750,35998252525,35998252525,True
3,4048211585,.1150903030,1150903030,False
4,4003224265,00000000000,,False
...,...,...,...,...
23480,4061585865,99999999999,99999999999,True
23481,4062895792,99999999999,99999999999,True
23482,4067616128,99999999999,99999999999,True
23483,4069286763,99999999999,99999999999,True


In [24]:
linhas[linhas["VALID_MSISDN"] == False]

,Conta: Código do cliente,Celular,Celular_fix,VALID_MSISDN
3,4048211585,.1150903030,1150903030,False
4,4003224265,00000000000,,False
5,4076487222,00000000000,,False
6,4090739624,00000000000,,False
7,4000159451,01121363994,1121363994,False
...,...,...,...,...
23279,4089163430,9291453022,9291453022,False
23373,4028787336,98534573204,98534573204,False
23442,4056720428,9997333799,9997333799,False
23454,4052049244,9999999999,9999999999,False


In [10]:
linhas[linhas["VALID_MSISDN"] == True].drop_duplicates("Celular_fix")[
    ["IMSI", "Celular_fix"]
]

,IMSI,Celular_fix
0,0,15998378377
1,0,35984286073
2,0,35998252525
18,0,11943040077
211,0,11900000000
...,...,...
23449,0,99991284868
23450,0,99991320297
23451,0,99991520325
23453,0,99991551951


In [11]:
linhas["IMSI"] = 0

linhas[linhas["VALID_MSISDN"] == True].drop_duplicates("Celular_fix")[
    ["IMSI", "Celular_fix"]
].to_csv(
    f"C:/Users/a0153041/Desktop/linhas_para_segmentar.csv",
    sep=",",
    decimal=".",
    encoding="utf_8",
    header=False,
    index=False,
    mode="w",
)

C:\Users\a0153041\AppData\Local\Temp\ipykernel_64228\2146644974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linhas["IMSI"] = 0


In [2]:
import pandas

files = [
    r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_08.parquet",
    r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_07.parquet",
    r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_06.parquet",
]
bases = []

agg_dict = {
    "Date (Call) YYYYMMDD": "nunique",
    "PMN (Settlement) Country": [
        "nunique",
        ("ListaPaís", (lambda x: set(x.to_list()))),
    ],
    "Charged MB": "sum",
    "MOC": "sum",
    "MTC": "sum",
    "Charged SMS": "sum",
}

for file in files:
    bases.append(
        pandas.read_parquet(
            r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_08.parquet",
            columns=[
                "Date (Call) YYYYMMDD",
                "PMN (Settlement) Country",
                "NR_TLFN",
                "Charged MB",
                "MOC",
                "MTC",
                "Charged SMS",
            ],
        )
    )


grouped_df = (
    pandas.concat(bases)
    .groupby(by=["NR_TLFN"], observed=True, dropna=False)
    .agg(agg_dict)
    .reset_index()
)

grouped_df.to_csv(
    r"c:/users/a0153041/desktop/trafego_three_months.csv",
    sep=";",
    decimal=",",
    encoding="utf_8",
    index=False,
)

In [33]:
agg_dict = {
    "Date (Call) YYYYMMDD": "nunique",
    "PMN (Settlement) Country": [
        "nunique",
        ("ListaPaís", (lambda x: set(x.to_list()))),
    ],
    "Charged MB": "sum",
    "MOC": "sum",
    "MTC": "sum",
    "Charged SMS": "sum",
}

grouped_df = (
    base.groupby(by=["NR_TLFN"], observed=True, dropna=False)
    .agg(agg_dict)
    .reset_index()
)

In [47]:
grouped_df.to_csv(
    r"c:/users/a0153041/desktop/trafego_ago.csv",
    sep=";",
    decimal=",",
    encoding="utf_8",
    index=False,
)

In [7]:
base = pandas.read_csv(
    r"c:/users/a0153041/desktop/trafego_three_months.csv",
    sep=";",
    decimal=",",
    encoding="utf_8",
    dtype={"NR_TLFN": str},
)

temp = pandas.merge(
    linhas[linhas["VALID_MSISDN"] == True].drop_duplicates("Celular_fix"),
    base,
    how="left",
    left_on="Celular_fix",
    right_on="NR_TLFN",
)
temp

C:\Users\a0153041\AppData\Local\Temp\ipykernel_69528\49959069.py:1: DtypeWarning: Columns (1,2,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  base = pandas.read_csv(


,Conta: Código do cliente,Celular,Celular_fix,VALID_MSISDN,NR_TLFN,Date (Call) YYYYMMDD,PMN (Settlement) Country,PMN (Settlement) Country.1,Charged MB,MOC,MTC,Charged SMS
0,4059104760,15998378377,15998378377,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4086584901,35984286073,35984286073,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4000907750,35998252525,35998252525,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4049707557,011943040077,11943040077,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4010760260,11900000000,11900000000,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9675,4048555775,99991284868,99991284868,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9676,4003866526,99991320297,99991320297,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9677,4002446305,99991520325,99991520325,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9678,4017360322,99991551951,99991551951,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
temp[temp["NR_TLFN"].notna()]

,Conta: Código do cliente,Celular,Celular_fix,VALID_MSISDN,IMSI,NR_TLFN,Date (Call) YYYYMMDD,PMN (Settlement) Country,PMN (Settlement) Country.1,Charged MB,MOC,MTC,Charged SMS
230,4060613478,11911042060,11911042060,True,0,11911042060,6,1,{'United States of America'},"13,121093753","0,0","0,0","0,0"
362,4002332886,11914214321,11914214321,True,0,11914214321,7,1,{'United States of America'},"3567,154296881","0,0","0,0","0,0"
385,4044074367,11915553785,11915553785,True,0,11915553785,4,1,{'Peru'},"1393,514820099","0,0","0,0","0,0"
494,4047673793,11919492928,11919492928,True,0,11919492928,6,2,"{'Panama', 'United States of America'}","1896,602539065","0,0","0,0","0,0"
631,4006225746,11932327333,11932327333,True,0,11932327333,3,1,{'Paraguay'},"302,984375003","0,0","2,0333333","0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23110,4005388226,85999989279,85999989279,True,0,85999989279,1,1,{'Argentina'},64.326172,0.0,0.0,0.0
23253,4004834784,91993404941,91993404941,True,0,91993404941,1,2,"{'Spain', 'Italy'}",439.34082,4.55,0.0,0.0
23301,4004672291,92984231400,92984231400,True,0,92984231400,8,3,"{'Panama', 'Aruba', 'Jamaica'}",3262.801758,0.0,0.316667,0.0
23362,4004932216,93991473668,93991473668,True,0,93991473668,22,2,"{'Spain', 'Portugal'}",14042.37207,0.0,0.0,0.0


In [8]:
temp.columns

Index(['Conta: Código do cliente', 'Celular', 'Celular_fix', 'VALID_MSISDN',
       'NR_TLFN', 'Date (Call) YYYYMMDD', 'PMN (Settlement) Country',
       'PMN (Settlement) Country.1', 'Charged MB', 'MOC', 'MTC',
       'Charged SMS'],
      dtype='object')

In [9]:
temp[
    [
        "Celular_fix",
        "Date (Call) YYYYMMDD",
        "PMN (Settlement) Country",
        "PMN (Settlement) Country.1",
        "Charged MB",
        "MOC",
        "MTC",
        "Charged SMS",
    ]
].to_excel(
    r"c:/users/a0153041/desktop/trafego_clevel.xlsx",
    index=False,
)

#### Extração DW

In [1]:
import calendar

YEAR = [2023, 2024, 2025]
for year in YEAR:
    for month in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
        print(calendar.monthrange(year, month)[1])

31
28
31
30
31
30
31
31
30
31
30
31
31
29
31
30
31
30
31
31
30
31
30
31
31
28
31
30
31
30
31
31
30
31
30
31


In [5]:
# Queries Teradata DW
import calendar

SET_QUERY_BAND = "SET QUERY_BAND = 'BLOCKCOMPRESSION=YES;' FOR SESSION;"
RMNG_OUT_MAI = "CREATE VOLATILE TABLE RMNG_OUT_MAI (IMSI BIGINT, MSISDN BIGINT) PRIMARY INDEX (MSISDN) ON COMMIT PRESERVE ROWS;"

START_DATE = f"{MES_TAP[:4]}-{MES_TAP[4:]}-01"  # "2025-09-01"
END_DATE = "2025-11-30"  # calendar.monthrange(int(MES_TAP[:4]), int(MES_TAP[4:]))[1]  # "2025-09-30"

print(START_DATE, END_DATE)

TB_TESTE_LINHAS_ID_PLNO = f"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_ID_PLNO AS (
SELECT
    A.IMSI,
    A.MSISDN AS NR_TLFN,
    B.DS_PRDT AS SEGMENTO,
    B.ID_PSSA,
    B.ID_LNHA,
    B.ID_PLNO,
    B.DT_FOTO_LNHA,
    B.ID_CNTA,
    B.CD_CNTA,
    B.DT_PRMR_ATVC_LNHA,
    CASE
        WHEN B.DS_MODL_ORIG NOT LIKE ALL
            (
                '%A%','%B%','%C%','%D%','%E%','%F%','%G%','%H%','%I%','%J%','%K%',
                '%L%','%M%','%N%','%O%','%P%','%Q%','%R%','%S%','%T%','%U%','%V%',
                '%W%','%X%','%Y%','%Z%','%#%','%/%','%@%','%\%','%-%','%.%','% %','%+%'
            )
            THEN Cast(B.DS_MODL_ORIG AS DECIMAL(8)) ELSE 0
    END AS NR_TAC
FROM RMNG_OUT_MAI A
LEFT JOIN P_VIEDB.VW_FAT_PRQE_LNHA_DSPT B
ON A.MSISDN = B.NR_TLFN
WHERE
    B.DT_FOTO_LNHA >= '{START_DATE}'
    AND B.DT_FOTO_LNHA <= '{END_DATE}'
    AND B.FL_PRQE_OFCL = 1
QUALIFY ROW_NUMBER() OVER (PARTITION BY A.MSISDN ORDER BY B.DT_FOTO_LNHA DESC) = 1
)
WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TB_TESTE_LINHAS_COMPLETO = r"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_COMPLETO AS(
SELECT
    A.IMSI,
    A.NR_TLFN,
    A.ID_LNHA,
    B.DS_PLNO,
    A.SEGMENTO,
    A.ID_PSSA,
    B.DS_CRCT_PLNO,
    A.DT_FOTO_LNHA,
    A.NR_TAC
FROM TB_TESTE_LINHAS_ID_PLNO A
LEFT JOIN P_VIEDB.VW_DIM_PLNO B
ON A.ID_PLNO = B.ID_PLNO
)
WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TB_TESTE_LINHAS_SRVC = f"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_SRVC, NO LOG AS(
SELECT 
    A.IMSI,
    A.NR_TLFN,
    A.ID_LNHA,
    A.DS_PLNO,
    A.SEGMENTO,
    A.ID_PSSA,
    A.DS_CRCT_PLNO,
    A.DT_FOTO_LNHA,
    B.ID_SRVC,
    B.DT_ATVC_SRVC,
    B.DT_DSTV_SRVC,
    A.NR_TAC
FROM TB_TESTE_LINHAS_COMPLETO A
LEFT JOIN P_VIEDB.VW_RLC_LNHA_SRVC B
ON A.ID_LNHA = B.ID_LNHA
    AND B.ID_SRVC IN ('9693','555655','638298','947427','1014701','111946',
    '1036745','1036743','635896','448429','1036741','1967','1014791','118476',
    '1014755','1014719','953522','137962','947425','947429','1036737','558000',
    '1014675','45077','581827','605170','953578','602975','485618','146521',
    '120612','953354','118489','953466','953298','947439','123604','947431',
    '517949','1014773','123649','483073','118475','1036739','111947','981',
    '953186','445826','668060','1014737','134108','579498','953242','1036747',
    '45078','947441','947435','953410','1036751','947433','123605','1014685',
    '520347','137963','1036749','128758','405865','1015235','1015239','1015236',
    '1015241','1015238','1015242','1015237','1015240')
    AND B.DT_ATVC_SRVC <= '{END_DATE}' -- '2023-08-31'
    AND B.DT_DSTV_SRVC > '{END_DATE}' -- '2023-08-31'
--GROUP BY 1,2,3,4,5
QUALIFY ROW_NUMBER() OVER (PARTITION BY A.NR_TLFN ORDER BY B.ID_SRVC DESC) = 1
)
WITH DATA PRIMARY INDEX(ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_TAC_V1 = r"""
CREATE VOLATILE TABLE TMP_DIM_TAC_V1, NO Log AS(
SELECT
    A.*,
    B.DS_FBRC_ORIG AS FABRICANTE,
    B.DS_MODL_ORIG AS MODELO,
    B.DS_TCNL,
    B.ID_TCNL,
    B.FL_DUAL_CHIP,
    B.FL_SMPH AS FL_SMARTPHONE,
    B.FL_SMPH_LITE AS FL_SMART_LITE,
    B.DS_CRCT_ESPC,
    B.DS_SIST_OPRC,
    B.NM_MODL_GSMA,
    B.DS_SIST_OPRC_GSMA,
    B.DS_CLSF_GSMA,
    B.DS_CLSF_SMPH,
    CASE
        WHEN B.DS_TCNL LIKE '%Band 28%' THEN 1
        WHEN B.DS_TCNL LIKE '%BAND 28%' THEN 1
    ELSE 0 END AS FL_700Mhz
FROM TB_TESTE_LINHAS_COMPLETO A
LEFT JOIN P_VIEDB.VW_DIM_TAC B
ON A.NR_TAC = B.NR_IMEI
--WHERE ID_MES = 202408
QUALIFY Row_Number() Over (PARTITION BY A.NR_TLFN ORDER BY ID_MES DESC) = 1
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_TAC_V2 = r"""
CREATE VOLATILE TABLE TMP_DIM_TAC_V2, NO Log AS(
SELECT
    A.*,
    CASE
        WHEN B.NM_TCNL = 'GSM' THEN '2G'
        WHEN B.NM_TCNL = 'WCDMA' THEN '3G'
        WHEN B.NM_TCNL = 'LTE' THEN '4G'
    ELSE B.NM_TCNL END AS TECNOLOGIA_APARELHO
FROM TMP_DIM_TAC_V1 A
LEFT JOIN P_VIEDB.VW_DIM_TCNL B
ON A.ID_TCNL = B.ID_TCNL
--QUALIFY Row_Number() Over (PARTITION BY A.NR_TAC ORDER BY A.NR_TAC) = 1
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_SRAL_LNHA = r"""
CREATE VOLATILE TABLE TMP_SRAL_LNHA, NO Log AS(
SELECT DISTINCT
    A.*,
    B.ID_TIPO_SRAL,
    B.NR_SRAL
FROM TMP_DIM_TAC_V2 A
LEFT JOIN P_VIEDB.VW_RLC_SRAL_LNHA B
ON A.ID_LNHA = B.ID_LNHA
WHERE (DT_VGNC_INCL <= (DATE - Extract(DAY From DATE)) AND DT_VGNC_FNAL > (DATE - Extract(DAY From DATE)))
QUALIFY Row_Number() Over(PARTITION BY A.NR_TLFN ORDER BY A.NR_TLFN, B.DT_VGNC_INCL DESC ) = 1
)
WITH DATA PRIMARY INDEX(NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_SRAL_V1 = r"""
CREATE VOLATILE TABLE TMP_DIM_SRAL_V1, NO Log AS(
SELECT
    A.*,
    ID_LNHA_PRMR_ATVC
FROM TMP_SRAL_LNHA A
LEFT JOIN P_VIEDB.VW_DIM_SRAL B
ON A.NR_SRAL = B.NR_SRAL
QUALIFY Row_Number() Over(PARTITION BY A.NR_TLFN ORDER BY DT_PRMR_ATVC_SRAL DESC ) = 1
)
WITH DATA PRIMARY INDEX(NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_PRQE_FINAL = r"""
CREATE VOLATILE TABLE TMP_PRQE_FINAL, NO Log AS(
SELECT
    A.IMSI,
    A.NR_TLFN,
    A.NR_SRAL,
    A.DS_CRCT_PLNO,
    A.DS_PLNO,
    A.SEGMENTO,
    CAST(OREPLACE(A.FABRICANTE,',',' -') AS VARCHAR(256)) AS FABRICANTE,
    CAST(OREPLACE(A.MODELO,',',' -') AS VARCHAR(256)) AS MODELO,
    A.DS_SIST_OPRC,
    A.DS_CLSF_GSMA,
    A.TECNOLOGIA_APARELHO,
    A.FL_DUAL_CHIP,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('01','02','03','04',
    '05','06','07','08','09','10','11','12','13','15','16','17','18','23',
    '25','26','27','29','30','31','32','37','40') THEN 1 ELSE 0 END AS FL_CHIP_3G,
    CASE 
        WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('14','19','21','28',
        '34','35','36','38','39','41','42','43','44','45','46','47','48',
        '49','50','51','52','53','54','55','56','57','58','59','60','62','66','67','71') THEN 1
        WHEN Substring(A.NR_SRAL From 1 FOR 6) = '895510' AND Substring(A.NR_SRAL From 10 FOR 2) IN ('92','93','94','95','96','97','98','99') THEN 1	
    ELSE 0 END AS FL_CHIP_4G,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('61','63','64','69') THEN 1 ELSE 0 END AS FL_CHIP_5G,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('52','54','55','62','63','64') THEN 1 ELSE 0 END AS FL_ESIM
FROM TMP_DIM_SRAL_V1 A
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

2025-11-01 2025-11-30


<>:62: SyntaxWarning: invalid escape sequence '\%'
<>:62: SyntaxWarning: invalid escape sequence '\%'
C:\Users\a0153041\AppData\Local\Temp\ipykernel_90328\122348871.py:62: SyntaxWarning: invalid escape sequence '\%'
  WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;


In [6]:
import teradatasql

# Exportar Informações para Teradata
TERADATA_PASSWORD = "Fevereiro32!"
TERADATA_HOST = "tdpvivo.redecorp.br"
TERADATA_USER = "A0153041"


def query_teradata(query, connection, cursor, verbose=True):
    try:
        sRequest = query
        cursor.execute(sRequest, ignoreErrors=3807)
        if verbose:
            # obtain the warnings and errors for transmitting the data to the database -- the acquisition phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + query
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + query
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = (
                "{fn teradata_nativesql}{fn teradata_logon_sequence_number}" + query
            )
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]
    finally:
        pass


def insert_csv_data(table_name, values: int, cursor, file, connection, verbose=True):
    insert_values = ""
    for i in range(values):
        insert_values += "?, "
    query = "INSERT INTO " + table_name + " (" + insert_values[:-2] + ")"
    print(query)

    try:
        connection.autocommit = False
        try:
            sInsert = "{fn teradata_read_csv(" + file + ")}" + query
            cursor.execute(sInsert)

            # obtain the warnings and errors for transmitting the data to the database -- the acquisition phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = (
                "{fn teradata_nativesql}{fn teradata_logon_sequence_number}" + sInsert
            )
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            connection.commit()

            # obtain the warnings and errors for the apply phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

        finally:
            connection.autocommit = True
    finally:
        if verbose:
            sRequest = f"SELECT TOP 5 * FROM {table_name}"
            print(sRequest)
            cursor.execute(sRequest, ignoreErrors=3807)
            print([row for row in cursor.fetchall()])


def export_full_table_to_csv(table_name, file_path, cursor):
    sRequest = f"SELECT * FROM {table_name}"
    sFileName = f"{file_path}/dataPy.csv"
    print(f"Exporting table data to file {sFileName}")
    try:
        os.makedirs(
            f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/02-LINHAS_SEGMENTADAS/{MES_TAP}/output_dw/"
        )
    except FileExistsError:
        ...
    except NameError:
        ...
    finally:
        cursor.execute("{fn teradata_write_csv(" + sFileName + ")}" + sRequest)
        print("File exporting was successful!")


file = f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/01-LINHAS_PARA_SEGMENTAR/{MES_TAP}/linhas_para_segmentar.csv"
folder = "C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/02-LINHAS_SEGMENTADAS"

# file = "C:/Users/a0153041/Desktop/linhas_para_segmentar.csv"
# folder = "C:/Users/a0153041/Desktop"

with teradatasql.connect(
    host=TERADATA_HOST,
    user=TERADATA_USER,
    password=TERADATA_PASSWORD,
    database="P_VIEDB",
    logmech="LDAP",
) as connect:
    with connect.cursor() as cur:
        query_teradata(SET_QUERY_BAND, connect, cur)
        query_teradata(RMNG_OUT_MAI, connect, cur)

        insert_csv_data("RMNG_OUT_MAI", 2, cur, file, connect)

        query_teradata(TB_TESTE_LINHAS_ID_PLNO, connect, cur)
        query_teradata(TB_TESTE_LINHAS_COMPLETO, connect, cur)
        query_teradata(TB_TESTE_LINHAS_SRVC, connect, cur)
        query_teradata(TMP_DIM_TAC_V1, connect, cur)
        query_teradata(TMP_DIM_TAC_V2, connect, cur)
        query_teradata(TMP_SRAL_LNHA, connect, cur)
        query_teradata(TMP_DIM_SRAL_V1, connect, cur)
        query_teradata(TMP_PRQE_FINAL, connect, cur)

        export_full_table_to_csv("TMP_PRQE_FINAL", f"{folder}", cur)

# Importar informações segmentadas
linhas_segmentadas = pandas.read_csv(
    # folder + f"/{MES_TAP}/output_dw/dataPy.csv",
    folder + f"/dataPy.csv",
    sep=",",
    encoding="utf_8",
    encoding_errors="ignore",
    on_bad_lines="skip",
    dtype={"IMSI": str, "NR_TLFN": str},
)
linhas_segmentadas.drop_duplicates(subset="IMSI", keep="first", inplace=True)

['']
['']
['']
['']
['']
['']
INSERT INTO RMNG_OUT_MAI (?, ?)
['']
['']
['']
['']
['']
SELECT TOP 5 * FROM RMNG_OUT_MAI
[[724235500590003, 33987497041], [724106201573098, 11957688542], [724116804416308, 86994826009], [724234293333287, 33998181599], [724115903251952, 21995913717]]
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
Exporting table data to file C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/02-LINHAS_SEGMENTADAS/dataPy.csv
File exporting was successful!


In [10]:
# Importar informações segmentadas
linhas_segmentadas = pandas.read_csv(
    folder + f"/{MES_TAP}/output_dw/dataPy.csv",
    # folder + f"/dataPy.csv",
    sep=",",
    encoding="utf_8",
    encoding_errors="ignore",
    on_bad_lines="skip",
    dtype={"IMSI": str, "NR_TLFN": str},
)
linhas_segmentadas.drop_duplicates(subset="IMSI", keep="first", inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/02-LINHAS_SEGMENTADAS/202509/output_dw/dataPy.csv'

## Geração da base final

In [7]:
print("Cruzando informações...")
# tarifas_cols = ['PMN (Settlement) TADIG Code', 'Tarifa_Dados', 'Tarifa_SMS', 'Tarifa_MOC', 'Tarifa_MTC']
# final = tap.merge(tarifas[tarifas_cols], on="PMN (Settlement) TADIG Code", how="left").merge(linhas_segmentadas, on="IMSI", how="left")
# tap = pandas.read_parquet(r'C:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\202507\final_segmentado.parquet')

# tap = pandas.read_csv(
#    r"c:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\202507\final_NACIONAL.csv.gz",
#    sep=";",
#    decimal=",",
#    encoding="utf_16",
# )

for col in ["PMN (Settlement) Country", "PMN (Settlement) TADIG Code", "Call Type"]:
    tap[col] = tap[col].astype("category")

for col in [
    "IMSI",
    "MSISDN",
    "Device TAC Code",
    "Number of Calls (incl. combined partials)",
    "Charged SMS",
    "Charged Minutes",
    "Charged MB",
    "Settlement Gross Charge - TAP Local Currency",  # "Settlement Charge - USD",
    # "Settlement Tax - USD",
    # "Settlement Gross Charge - USD",
    # "Settlement Gross Charge - EUR",
]:
    try:
        tap[col] = pandas.to_numeric(tap[col], downcast="float")
    except KeyError:
        continue
    except ValueError:
        tap[col] = tap[col].astype(str).replace("nan", 0)
        tap[col] = pandas.to_numeric(tap[col], downcast="float")

tap["MSISDN_fix"] = tap["MSISDN_fix"].astype(str).str.replace(".0", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].str.replace("<NA>", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].astype("category")

tap["IMSI"] = tap["IMSI"].astype(str).str.replace(".0", "")
tap["IMSI"] = tap["IMSI"].astype(str).str.replace("nan", "")
tap["IMSI"] = tap["IMSI"].str.replace("<NA>", "")
tap["IMSI"] = tap["IMSI"].astype("category")


# Importar informações segmentadas
folder = "C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/02-LINHAS_SEGMENTADAS"
linhas_segmentadas = pandas.read_csv(
    folder + f"/dataPy.csv",
    sep=",",
    encoding="utf_8",
    encoding_errors="ignore",
    on_bad_lines="skip",
    dtype={"IMSI": str, "NR_TLFN": str},
)
linhas_segmentadas.drop_duplicates(subset="IMSI", keep="first", inplace=True)

linhas_segmentadas["IMSI"] = linhas_segmentadas["IMSI"].astype(str)
tap["IMSI"] = tap["IMSI"].astype(str)

final = tap.merge(linhas_segmentadas, on="IMSI", how="left")

# del tarifas
del linhas_segmentadas
del tap


def add_values_to_dataframe(conditions: list, values: list, default=0) -> pandas.Series:
    return numpy.select(conditions, values, default=default)


print("Ajustando colunas...")
"""
conditions = [
    final["Call Type"] == "MOC",
    final["Call Type"] == "MTC",
    final["Call Type"] == "SMS-MO",
    final["Call Type"] == "GPRS",
]
values = [
    final["Tarifa_MOC"].astype(float) * final["Charged Minutes"],
    final["Tarifa_MTC"].astype(float) * final["Charged Minutes"],
    final["Tarifa_SMS"].astype(float) * final["Charged Minutes"],
    final["Tarifa_Dados"].astype(float) * final["Charged MB"]
]
"""
# final["Custo_USD"] = numpy.select(conditions, values, default=0)
# final["Custo_USD"] = add_values_to_dataframe(conditions, values, default=0)

conditions = [
    final["SEGMENTO"].str.startswith("PF_POS") == True,
    final["SEGMENTO"].str.startswith("PF_RESIDENCIAL") == True,
    final["SEGMENTO"].str.startswith("PF_PRE") == True,
    final["SEGMENTO"].str.startswith("PF_CONTROLE") == True,
    final["SEGMENTO"].str.startswith("PJ") == True,
]
values = ["B2C_POS", "B2C_POS", "B2C_PRE", "B2C_CONTROLE", "B2B"]
final["SEGMENTO_fix"] = add_values_to_dataframe(conditions, values, default=pandas.NA)

# Atribuir os tipos corretos e excluir colunas desnecessárias
cols = [
    "PMN (Settlement) Country",
    "PMN (Settlement) TADIG Code",
    "Call Type",
    "IMSI",
    "MSISDN",
    "MSISDN_fix",
    "NR_TLFN",
    "NR_SRAL",
    "DS_CRCT_PLNO",
    "DS_PLNO",
    "SEGMENTO",
    "FABRICANTE",
    "MODELO",
    "DS_SIST_OPRC",
    "DS_CLSF_GSMA",
    "TECNOLOGIA_APARELHO",
    "SEGMENTO_fix",
    "FL_DUAL_CHIP",
    "FL_CHIP_3G",
    "FL_CHIP_4G",
    "FL_CHIP_5G",
    "FL_ESIM",
]
for col in cols:
    try:
        final[col] = final[col].astype("category")
    except KeyError:
        continue

print("Criando colunas para análise de diárias...")
# Para Diárias
final["MOC"] = final[final["Call Type"] == "MOC"]["Charged Minutes"]
final["MOC"] = final["MOC"].astype(str).replace("nan", "0").astype(float)

final["MTC"] = final[final["Call Type"] == "MTC"]["Charged Minutes"]
final["MTC"] = final["MTC"].astype(str).replace("nan", "0").astype(float)

final["Voz"] = final["MOC"] + final["MTC"]

dados_utilizados = (
    final.groupby(by=["SEGMENTO_fix"])
    .agg({"Charged MB": "sum", "IMSI": "nunique"})
    .reset_index()
)
dados_utilizados["Charged TB"] = dados_utilizados["Charged MB"] / 1024**2
print(dados_utilizados)
print(
    "Não Identificado: ",
    final[final["SEGMENTO_fix"].isna()]["IMSI"].nunique(),
    " IMSIs",
)
print(
    "Não Identificado: ",
    final[final["SEGMENTO_fix"].isna()]["Charged MB"].sum() / 1024**2,
    " TB",
)


# print(final["Settlement Gross Charge - TAP Local Currency"].sum())
final.to_parquet(str(BASE_TAP) + f"/{MES_TAP}.parquet")

Cruzando informações...
Ajustando colunas...
Criando colunas para análise de diárias...
   SEGMENTO_fix    Charged MB     IMSI  Charged TB
0           B2B  2.655406e+07   413860   25.323928
1  B2C_CONTROLE  3.733610e+07  1022259   35.606479
2       B2C_POS  5.386774e+07   639947   51.372278
3       B2C_PRE  1.414955e+07   477381   13.494064
Não Identificado:  47637  IMSIs
Não Identificado:  1.239792119026184  TB


### Big Numbers

In [ ]:
dados_utilizados = (
    final[final["SEGMENTO_fix"].notnull()]
    .groupby(by=["SEGMENTO_fix"], observed=True, dropna=False)
    .agg({"Charged MB": "sum", "IMSI": "nunique"})
    .reset_index()
)

dados_utilizados["Charged TB"] = dados_utilizados["Charged MB"] / 1024**2

dados_utilizados["perc_tb"] = round(
    dados_utilizados["Charged TB"] / dados_utilizados["Charged TB"].sum() * 100, 2
)

dados_utilizados["perc_users"] = round(
    dados_utilizados["IMSI"] / dados_utilizados["IMSI"].sum() * 100, 2
)

# dados_utilizados[["SEGMENTO_fix", "Charged TB", "IMSI", "perc_tb", "perc_users"]]

media_viagem = (
    final[final["SEGMENTO_fix"].notnull()]
    .groupby(by=["SEGMENTO_fix", "IMSI"], observed=True, dropna=False)
    .agg({"Date (Call)": "nunique"})
    .reset_index()
    .groupby(by=["SEGMENTO_fix"], observed=True, dropna=False)
    .agg({"Date (Call)": "mean"})
    .reset_index()
)

media_viagem.rename(columns={"Date (Call)": "Dias em Viagem"}, inplace=True)

# media_viagem

travel_used = (
    final[final["SEGMENTO_fix"].notnull()]
    .groupby(by=["SEGMENTO_fix", "IMSI"], observed=True, dropna=False)
    .agg(
        {
            "Charged Events": "sum",
            "Charged Minutes": "sum",
            "Charged MB": "sum",
        }
    )
    .reset_index()
)

travel_used = travel_used[
    (travel_used["Charged MB"] >= 1)
    | (travel_used["Charged Events"] > 0)
    | (travel_used["Charged Minutes"] > 0)
]

travel_used = (
    travel_used.groupby(by=["SEGMENTO_fix"], observed=True, dropna=False)
    .agg({"IMSI": "nunique"})
    .reset_index()
)
travel_used.rename(columns={"IMSI": "travel_used"}, inplace=True)

# travel_used

big_numbers = dados_utilizados.merge(media_viagem, how="left", on="SEGMENTO_fix").merge(
    travel_used, how="left", on="SEGMENTO_fix"
)

big_numbers["perc_travel_util"] = round(
    big_numbers["travel_used"] / big_numbers["IMSI"] * 100, 2
)
big_numbers["Dias em Viagem"] = round(big_numbers["Dias em Viagem"], 2)

big_numbers[
    [
        "SEGMENTO_fix",
        "Dias em Viagem",
        "IMSI",
        "perc_travel_util",
        "Charged TB",
        "perc_tb",
        "perc_users",
    ]
]

,SEGMENTO_fix,Dias em Viagem,IMSI,perc_travel_util,Charged TB,perc_tb,perc_users
0,B2B,8.16,94854,82.30,341.580108,24.33,11.29
1,B2C_CONTROLE,5.75,238293,38.06,43.137575,3.07,28.35
2,B2C_POS,8.79,342194,82.09,1010.897994,71.99,40.71
3,B2C_PRE,7.96,165137,45.81,8.537919,0.61,19.65


In [ ]:
big_numbers[big_numbers["SEGMENTO_fix"].notnull()][
    [
        "SEGMENTO_fix",
        "IMSI",
        "Charged TB",
        "perc_users",
        "perc_travel_util",
        "perc_tb",
        "Dias em Viagem",
    ]
].to_excel(
    "c:/users/a0153041/desktop/resumo_roaming_internacional_202511.xlsx", index=False
)

In [ ]:
import pandas

final = pandas.read_parquet(
    r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2024_09.parquet"
)

dados_utilizados = (
    final[final["SEGMENTO_fix"].notnull()]
    .groupby(by=["SEGMENTO_fix"], observed=True, dropna=False)
    .agg({"Charged MB": "sum", "IMSI": "nunique"})
    .reset_index()
)

dados_utilizados["Charged TB"] = dados_utilizados["Charged MB"] / 1024**2

dados_utilizados["perc_tb"] = round(
    dados_utilizados["Charged TB"] / dados_utilizados["Charged TB"].sum() * 100, 2
)

dados_utilizados["perc_users"] = round(
    dados_utilizados["IMSI"] / dados_utilizados["IMSI"].sum() * 100, 2
)

# dados_utilizados[["SEGMENTO_fix", "Charged TB", "IMSI", "perc_tb", "perc_users"]]

media_viagem = (
    final[final["SEGMENTO_fix"].notnull()]
    .groupby(by=["SEGMENTO_fix", "IMSI"], observed=True, dropna=False)
    .agg({"Date (Call) YYYYMMDD": "nunique"})
    .reset_index()
    .groupby(by=["SEGMENTO_fix"], observed=True, dropna=False)
    .agg({"Date (Call) YYYYMMDD": "mean"})
    .reset_index()
)

media_viagem.rename(columns={"Date (Call) YYYYMMDD": "Dias em Viagem"}, inplace=True)

# media_viagem

travel_used = (
    final[final["SEGMENTO_fix"].notnull()]
    .groupby(by=["SEGMENTO_fix", "IMSI"], observed=True, dropna=False)
    .agg(
        {
            "Charged SMS": "sum",
            "Charged Minutes": "sum",
            "Charged MB": "sum",
        }
    )
    .reset_index()
)

travel_used = travel_used[
    (travel_used["Charged MB"] >= 1)
    | (travel_used["Charged SMS"] > 0)
    | (travel_used["Charged Minutes"] > 0)
]

travel_used = (
    travel_used.groupby(by=["SEGMENTO_fix"], observed=True, dropna=False)
    .agg({"IMSI": "nunique"})
    .reset_index()
)
travel_used.rename(columns={"IMSI": "travel_used"}, inplace=True)

# travel_used

big_numbers_set_py = dados_utilizados.merge(
    media_viagem, how="left", on="SEGMENTO_fix"
).merge(travel_used, how="left", on="SEGMENTO_fix")

big_numbers_set_py["perc_travel_util"] = round(
    big_numbers_set_py["travel_used"] / big_numbers_set_py["IMSI"] * 100, 2
)
big_numbers_set_py["Dias em Viagem"] = round(big_numbers_set_py["Dias em Viagem"], 2)

big_numbers_set_py[
    [
        "SEGMENTO_fix",
        "Dias em Viagem",
        "IMSI",
        "perc_travel_util",
        "Charged TB",
        "perc_tb",
        "perc_users",
    ]
]

KeyError: "Column(s) ['Charged Events'] do not exist"

In [ ]:
travel_used = (
    final[final["SEGMENTO_fix"].notnull()]
    .groupby(by=["SEGMENTO_fix", "IMSI"], observed=True, dropna=False)
    .agg(
        {
            "Charged SMS": "sum",
            "Charged Minutes": "sum",
            "Charged MB": "sum",
        }
    )
    .reset_index()
)

travel_used = travel_used[
    (travel_used["Charged MB"] >= 1)
    | (travel_used["Charged SMS"] > 0)
    | (travel_used["Charged Minutes"] > 0)
]

travel_used = (
    travel_used.groupby(by=["SEGMENTO_fix"], observed=True, dropna=False)
    .agg({"IMSI": "nunique"})
    .reset_index()
)
travel_used.rename(columns={"IMSI": "travel_used"}, inplace=True)

# travel_used

big_numbers_set_py = dados_utilizados.merge(
    media_viagem, how="left", on="SEGMENTO_fix"
).merge(travel_used, how="left", on="SEGMENTO_fix")

big_numbers_set_py["perc_travel_util"] = round(
    big_numbers_set_py["travel_used"] / big_numbers_set_py["IMSI"] * 100, 2
)
big_numbers_set_py["Dias em Viagem"] = round(big_numbers_set_py["Dias em Viagem"], 2)

big_numbers_set_py[
    [
        "SEGMENTO_fix",
        "Dias em Viagem",
        "IMSI",
        "perc_travel_util",
        "Charged TB",
        "perc_tb",
        "perc_users",
    ]
]

,SEGMENTO_fix,Dias em Viagem,IMSI,perc_travel_util,Charged TB,perc_tb,perc_users
0,B2B,8.62,78634,85.97,217.499269,20.52,10.72
1,B2C_CONTROLE,5.63,202579,37.81,8.967935,0.85,27.63
2,B2C_POS,9.45,283086,86.89,829.915837,78.31,38.61
3,B2C_PRE,6.95,168958,42.51,3.412368,0.32,23.04


In [4]:
big_numbers_set_py[big_numbers_set_py["SEGMENTO_fix"].notnull()][
    [
        "SEGMENTO_fix",
        "IMSI",
        "Charged TB",
        "perc_users",
        "perc_travel_util",
        "perc_tb",
        "Dias em Viagem",
    ]
].to_excel(
    "c:/users/a0153041/desktop/resumo_roaming_internacional_set_24.xlsx", index=False
)

## CdG

### Internacional

In [15]:
import os
import pathlib

paths = sorted(
    pathlib.Path(
        r"C:\Users\a0153041\Telefonica\Roaming - General\Financeiro\04 - Tarifas"
    ).iterdir(),
    key=os.path.getmtime,
)
iot_paths = []
for i, v in enumerate(paths):
    if "TARIFAS_ATUAIS" in str(v):
        iot_paths.append(v)
iot_path = sorted(iot_paths)[0]

try:
    cdg = (
        final.groupby(
            by=[
                "PMN (Settlement) TADIG Code",
                "PMN (Settlement) Country",
                "Date (Call) YYYYMMDD",
                "Call Type",
                "IMSI",
                "NR_TLFN",
                "SEGMENTO",
            ],
            observed=True,
        )
        .agg({"Charged SMS": "sum", "Charged MB": "sum", "MOC": "sum", "MTC": "sum"})
        .reset_index()
    )
except Exception as e:
    import pandas
    import numpy

    MES_TAP = "202510"
    final = pandas.read_parquet(
        r"C:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_10.parquet",
        columns=[
            "PMN (Settlement) TADIG Code",
            "PMN (Settlement) Country",
            "Date (Call)",
            "Call Type",
            "IMSI",
            "NR_TLFN",
            "SEGMENTO",
            "Charged Events",
            "Charged MB",
            "MOC",
            "MTC",
            "SEGMENTO_fix",
        ],
        # filters=[
        #    [('SEGMENTO_fix', '=', 'B2C_POS')],
        #    [('SEGMENTO_fix', '=', 'B2C_PRE')],
        #    [('SEGMENTO_fix', '=', 'B2C_CONTROLE')]
        # ]
    )

    cdg = (
        final.groupby(
            by=[
                "PMN (Settlement) TADIG Code",
                "PMN (Settlement) Country",
                "Date (Call)",
                "Call Type",
                "IMSI",
                "NR_TLFN",
                "SEGMENTO",
            ],
            observed=True,
        )
        .agg({"Charged Events": "sum", "Charged MB": "sum", "MOC": "sum", "MTC": "sum"})
        .reset_index()
    )
    del final

tarifas = pandas.read_excel(iot_path)
tarifas.rename(columns={"PLMN": "PMN (Settlement) TADIG Code"}, inplace=True)

cols = ["ACORDO_MOC", "ACORDO_MTC", "ACORDO_SMS_MO", "ACORDO_GPRS"]
for col in cols:
    tarifas[tarifas[col] == "Gross"] = 0.0
    tarifas[col] = pandas.to_numeric(tarifas[col], downcast="float", errors="raise")

cdg = cdg.merge(tarifas, how="left", on="PMN (Settlement) TADIG Code")

cdg["PMN (Settlement) TADIG Code"] = cdg["PMN (Settlement) TADIG Code"].astype(
    "category"
)
cdg["ALPHA_3"] = cdg["ALPHA_3"].astype("category")
cdg["short_name_ptbr"] = cdg["short_name_ptbr"].astype("category")
cdg["region"] = cdg["region"].astype("category")
cdg["sub-region"] = cdg["sub-region"].astype("category")
cdg["MOEDA"] = cdg["MOEDA"].astype("category")

cdg["Date (Call)"] = pandas.to_numeric(cdg["Date (Call)"], downcast="unsigned")

cdg["CN"] = cdg["NR_TLFN"].astype(str).str[0:2]
cdg["CN"] = cdg["CN"].astype("category")

cdg["MOC (Total)"] = cdg["ACORDO_MOC"] * cdg["MOC"]
cdg["MTC (Total)"] = cdg["ACORDO_MTC"] * cdg["MTC"]
cdg["GPRS (Total)"] = cdg["Charged MB"] * cdg["ACORDO_GPRS"]
cdg["SMS-MO (Total)"] = cdg["Charged Events"] * cdg["ACORDO_SMS_MO"]
cdg["Total"] = (
    cdg["MOC (Total)"]
    + cdg["MTC (Total)"]
    + cdg["GPRS (Total)"]
    + cdg["SMS-MO (Total)"]
)

cols = [
    "PMN (Settlement) TADIG Code",
    "PMN (Settlement) Country",
    "Date (Call)",
    "Call Type",
    "IMSI",
    "MSISDN",
    "Charged Events",
    "Charged Minutes",
    "Charged MB",
    "SEGMENTO" "MOC_x",
    "MTC_x",
    "CN",
    "MOC",
    "MTC",
    "SMS-MO",
    "GPRS",
    "MOC (Total)",
    "MTC (Total)",
    "GPRS (Total)",
    "SMS-MO (Total)",
    "Total",
]


class bcolors:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"


def exportSplittedReport(dataframe: pandas.DataFrame, exportTo: str):
    with pandas.ExcelWriter(exportTo + ".xlsx", engine="xlsxwriter") as writer:
        ROW_LIMIT = 1048576  # Excel row limit

        if len(dataframe) > ROW_LIMIT:
            chunks = (len(dataframe) // ROW_LIMIT) + 1
            print(f"{bcolors.OKCYAN}Chunks: {chunks}{bcolors.ENDC}")

            # saves each chunk to a different sheet
            for i, chunk in enumerate(numpy.array_split(dataframe, chunks)):
                temp_sheet = f"Dados-Parte_{i+1}"  # name of the sheet

                print(
                    f"{bcolors.OKCYAN}Exportando planilha: {temp_sheet}...{bcolors.ENDC}"
                )
                chunk.to_excel(writer, sheet_name=temp_sheet, index=False)
                print(
                    f"{bcolors.OKGREEN}Planilha {temp_sheet} exportada com sucesso!{bcolors.ENDC}"
                )
        else:
            dataframe.to_excel(writer, sheet_name="Dados", index=False)
    print(f"Saved to: {exportTo}.xlsx")


cdg.groupby(by=["CN", "SEGMENTO"], observed=True, dropna=False).agg(
    {
        "GPRS (Total)": "sum",
        "SMS-MO (Total)": "sum",
        "MOC (Total)": "sum",
        "MTC (Total)": "sum",
        "Total": "sum",
    }
).reset_index().to_excel(
    f"C:/Users/a0153041/Telefonica/Roaming - General/DMVNO-C/22 - Bases/CDG/Internacional/Resumos/{MES_TAP}_Internacional.xlsx",
    index=False,
)

### Nacional

In [8]:
import pandas

CURRENCY_PRECISION = 6
CURRENCY_COLS = ["MOC (Total)", "MTC (Total)", "GPRS (Total)", "SMS-MO (Total)"]
CURRENCY_COLUMN = "Settlement Gross Charge - TAP Local Currency"
TARIFFS = {
    "TARIFA_GPRS": "Charged MB",
    "TARIFA_MOC": "MOC",
    "TARIFA_MTC": "MTC",
    "TARIFA_SMS": "Charged Events",
}
COSTS = {
    "GPRS (Total)": ["TARIFA_GPRS", "Charged MB"],
    "SMS-MO (Total)": ["TARIFA_SMS", "Charged Events"],
    "MOC (Total)": ["TARIFA_MOC", "MOC"],
    "MTC (Total)": ["TARIFA_MTC", "MTC"],
}

try:
    cdg = (
        final.groupby(
            by=[
                "PMN (Settlement) TADIG Code",
                "PMN (Settlement) Country",
                "Date (Call)",
                "Call Type",
                "IMSI",
                "NR_TLFN",
                "SEGMENTO",
            ],
            observed=True,
        )
        .agg(
            {
                "Charged Events": "sum",
                "Charged MB": "sum",
                "MOC": "sum",
                "MTC": "sum",
                "Settlement Gross Charge - TAP Local Currency": "sum",
            }
        )
        .reset_index()
    )
except NameError:
    final = pandas.read_parquet(str(BASE_TAP) + f"/{MES_TAP}_NACIONAL.parquet")
    cdg = (
        final.groupby(
            by=[
                "PMN (Settlement) TADIG Code",
                "PMN (Settlement) Country",
                "Date (Call)",
                "Call Type",
                "IMSI",
                "NR_TLFN",
                "SEGMENTO",
            ],
            observed=True,
        )
        .agg(
            {
                "Charged Events": "sum",
                "Charged MB": "sum",
                "MOC": "sum",
                "MTC": "sum",
                "Settlement Gross Charge - TAP Local Currency": "sum",
            }
        )
        .reset_index()
    )
finally:
    cdg["CN"] = cdg["NR_TLFN"].astype(str).str[0:2]

    # Calculate tariffs as [charged_total / charged_qty]
    # cdg["TARIFA_GPRS"] = round(
    #    cdg[CURRENCY_COLUMN] / cdg["Charged MB"],
    #    CURRENCY_PRECISION,
    # )
    for k, v in enumerate(TARIFFS):
        cdg[v] = round(
            cdg[CURRENCY_COLUMN] / cdg[TARIFFS[v]],
            CURRENCY_PRECISION,
        )

    # Calculate costs as [tariff * charged qty]
    #  cdg["MOC (Total)"] = round(cdg["TARIFA_MOC"] * cdg["MOC"], CURRENCY_PRECISION)
    for k, v in enumerate(COSTS):
        cdg[v] = round(cdg[COSTS[v][0]] * cdg[COSTS[v][1]], CURRENCY_PRECISION)

    temp = (
        cdg.groupby(by=["CN", "SEGMENTO"], observed=True, dropna=False)
        .agg(
            {
                "GPRS (Total)": "sum",
                "SMS-MO (Total)": "sum",
                "MOC (Total)": "sum",
                "MTC (Total)": "sum",
            }
        )
        .reset_index()
    )

    for col in CURRENCY_COLS:
        temp[col] = round(temp[col], CURRENCY_PRECISION)

    temp["Total"] = round(
        temp["GPRS (Total)"]
        + temp["SMS-MO (Total)"]
        + temp["MOC (Total)"]
        + temp["MTC (Total)"],
        CURRENCY_PRECISION,
    )

    temp.to_excel(
        f"C:\\Users\\a0153041\\Telefonica\\Roaming - General\\DMVNO-C\\22 - Bases\\CDG\\Nacional\\Resumos\\{MES_TAP}_Nacional.xlsx",
        # f"C:\\Users\\a0153041\\Desktop\\{MES_TAP}_Nacional_v3.xlsx",
        index=False,
    )

In [9]:
resumo = (
    final.groupby(by=["SEGMENTO_fix"], observed=True, dropna=False)
    .agg({"Settlement Gross Charge - TAP Local Currency": "sum"})
    .reset_index()
)
resumo["Custo"] = round(resumo["Settlement Gross Charge - TAP Local Currency"], 2)
resumo

,SEGMENTO_fix,Settlement Gross Charge - TAP Local Currency,Custo
0,B2B,173604.515625,173604.515625
1,B2C_CONTROLE,156807.796875,156807.796875
2,B2C_POS,311671.468750,311671.468750
3,B2C_PRE,68195.500000,68195.500000
4,NaN,7370.629883,7370.629883


## Restabelecimento de Velocidade

In [ ]:
# Queries Teradata DW

SET_QUERY_BAND = "SET QUERY_BAND = 'BLOCKCOMPRESSION=YES;' FOR SESSION;"
RMNG_OUT_MAI = "CREATE VOLATILE TABLE RMNG_OUT_MAI (IMSI BIGINT, MSISDN BIGINT) PRIMARY INDEX (MSISDN) ON COMMIT PRESERVE ROWS;"

TB_TESTE_LINHAS_ID_PLNO = r"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_ID_PLNO AS (
SELECT
    A.IMSI,
    A.MSISDN AS NR_TLFN,
    B.DS_PRDT AS SEGMENTO,
    B.ID_PSSA,
    B.ID_LNHA,
    B.ID_PLNO,
    B.DT_FOTO_LNHA,
    B.ID_CNTA,
    B.CD_CNTA,
    B.DT_PRMR_ATVC_LNHA,
    CASE
        WHEN B.DS_MODL_ORIG NOT LIKE ALL
            (
                '%A%','%B%','%C%','%D%','%E%','%F%','%G%','%H%','%I%','%J%','%K%',
                '%L%','%M%','%N%','%O%','%P%','%Q%','%R%','%S%','%T%','%U%','%V%',
                '%W%','%X%','%Y%','%Z%','%#%','%/%','%@%','%\%','%-%','%.%','% %','%+%'
            )
            THEN Cast(B.DS_MODL_ORIG AS DECIMAL(8)) ELSE 0
    END AS NR_TAC
FROM RMNG_OUT_MAI A
LEFT JOIN P_VIEDB.VW_FAT_PRQE_LNHA_DSPT B
ON A.MSISDN = B.NR_TLFN
WHERE
    B.DT_FOTO_LNHA >= '2024-08-01'
    AND B.DT_FOTO_LNHA <= '2024-08-31'
    AND B.FL_PRQE_OFCL = 1
QUALIFY ROW_NUMBER() OVER (PARTITION BY A.MSISDN ORDER BY B.DT_FOTO_LNHA DESC) = 1
)
WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TB_TESTE_LINHAS_COMPLETO = r"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_COMPLETO AS(
SELECT
    A.IMSI,
    A.NR_TLFN,
    A.ID_LNHA,
    B.DS_PLNO,
    A.SEGMENTO,
    A.ID_PSSA,
    B.DS_CRCT_PLNO,
    A.DT_FOTO_LNHA,
    A.NR_TAC
FROM TB_TESTE_LINHAS_ID_PLNO A
LEFT JOIN P_VIEDB.VW_DIM_PLNO B
ON A.ID_PLNO = B.ID_PLNO
)
WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TB_TESTE_LINHAS_SRVC = r"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_SRVC, NO LOG AS(
SELECT 
    A.IMSI,
    A.NR_TLFN,
    A.ID_LNHA,
    A.DS_PLNO,
    A.SEGMENTO,
    A.ID_PSSA,
    A.DS_CRCT_PLNO,
    A.DT_FOTO_LNHA,
    B.ID_SRVC,
    B.DT_ATVC_SRVC,
    B.DT_DSTV_SRVC,
    A.NR_TAC
FROM TB_TESTE_LINHAS_COMPLETO A
LEFT JOIN P_VIEDB.VW_RLC_LNHA_SRVC B
ON A.ID_LNHA = B.ID_LNHA
    AND B.ID_SRVC IN ('9693','555655','638298','947427','1014701','111946',
    '1036745','1036743','635896','448429','1036741','1967','1014791','118476',
    '1014755','1014719','953522','137962','947425','947429','1036737','558000',
    '1014675','45077','581827','605170','953578','602975','485618','146521',
    '120612','953354','118489','953466','953298','947439','123604','947431',
    '517949','1014773','123649','483073','118475','1036739','111947','981',
    '953186','445826','668060','1014737','134108','579498','953242','1036747',
    '45078','947441','947435','953410','1036751','947433','123605','1014685',
    '520347','137963','1036749','128758','405865','1015235','1015239','1015236',
    '1015241','1015238','1015242','1015237','1015240')
    AND B.DT_ATVC_SRVC <= '2024-08-31' -- '2023-08-31'
    AND B.DT_DSTV_SRVC > '2024-08-31' -- '2023-08-31'
--GROUP BY 1,2,3,4,5
QUALIFY ROW_NUMBER() OVER (PARTITION BY A.NR_TLFN ORDER BY B.ID_SRVC DESC) = 1
)
WITH DATA PRIMARY INDEX(ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_TAC_V1 = r"""
CREATE VOLATILE TABLE TMP_DIM_TAC_V1, NO Log AS(
SELECT
    A.*,
    B.DS_FBRC_ORIG AS FABRICANTE,
    B.DS_MODL_ORIG AS MODELO,
    B.DS_TCNL,
    B.ID_TCNL,
    B.FL_DUAL_CHIP,
    B.FL_SMPH AS FL_SMARTPHONE,
    B.FL_SMPH_LITE AS FL_SMART_LITE,
    B.DS_CRCT_ESPC,
    B.DS_SIST_OPRC,
    B.NM_MODL_GSMA,
    B.DS_SIST_OPRC_GSMA,
    B.DS_CLSF_GSMA,
    B.DS_CLSF_SMPH,
    CASE
        WHEN B.DS_TCNL LIKE '%Band 28%' THEN 1
        WHEN B.DS_TCNL LIKE '%BAND 28%' THEN 1
    ELSE 0 END AS FL_700Mhz
FROM TB_TESTE_LINHAS_COMPLETO A
LEFT JOIN P_VIEDB.VW_DIM_TAC B
ON A.NR_TAC = B.NR_IMEI
--WHERE ID_MES = 202408
QUALIFY Row_Number() Over (PARTITION BY A.NR_TLFN ORDER BY ID_MES DESC) = 1
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_TAC_V2 = r"""
CREATE VOLATILE TABLE TMP_DIM_TAC_V2, NO Log AS(
SELECT
    A.*,
    CASE
        WHEN B.NM_TCNL = 'GSM' THEN '2G'
        WHEN B.NM_TCNL = 'WCDMA' THEN '3G'
        WHEN B.NM_TCNL = 'LTE' THEN '4G'
    ELSE B.NM_TCNL END AS TECNOLOGIA_APARELHO
FROM TMP_DIM_TAC_V1 A
LEFT JOIN P_VIEDB.VW_DIM_TCNL B
ON A.ID_TCNL = B.ID_TCNL
--QUALIFY Row_Number() Over (PARTITION BY A.NR_TAC ORDER BY A.NR_TAC) = 1
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_SRAL_LNHA = r"""
CREATE VOLATILE TABLE TMP_SRAL_LNHA, NO Log AS(
SELECT DISTINCT
    A.*,
    B.ID_TIPO_SRAL,
    B.NR_SRAL
FROM TMP_DIM_TAC_V2 A
LEFT JOIN P_VIEDB.VW_RLC_SRAL_LNHA B
ON A.ID_LNHA = B.ID_LNHA
WHERE (DT_VGNC_INCL <= (DATE - Extract(DAY From DATE)) AND DT_VGNC_FNAL > (DATE - Extract(DAY From DATE)))
QUALIFY Row_Number() Over(PARTITION BY A.NR_TLFN ORDER BY A.NR_TLFN, B.DT_VGNC_INCL DESC ) = 1
)
WITH DATA PRIMARY INDEX(NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_SRAL_V1 = r"""
CREATE VOLATILE TABLE TMP_DIM_SRAL_V1, NO Log AS(
SELECT
    A.*,
    ID_LNHA_PRMR_ATVC
FROM TMP_SRAL_LNHA A
LEFT JOIN P_VIEDB.VW_DIM_SRAL B
ON A.NR_SRAL = B.NR_SRAL
QUALIFY Row_Number() Over(PARTITION BY A.NR_TLFN ORDER BY DT_PRMR_ATVC_SRAL DESC ) = 1
)
WITH DATA PRIMARY INDEX(NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_PRQE_FINAL = r"""
CREATE VOLATILE TABLE TMP_PRQE_FINAL, NO Log AS(
SELECT
    A.IMSI,
    A.NR_TLFN,
    A.NR_SRAL,
    A.DS_CRCT_PLNO,
    A.DS_PLNO,
    A.SEGMENTO,
    CAST(OREPLACE(A.FABRICANTE,',',' -') AS VARCHAR(256)) AS FABRICANTE,
    CAST(OREPLACE(A.MODELO,',',' -') AS VARCHAR(256)) AS MODELO,
    A.DS_SIST_OPRC,
    A.DS_CLSF_GSMA,
    A.TECNOLOGIA_APARELHO,
    A.FL_DUAL_CHIP,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('01','02','03','04',
    '05','06','07','08','09','10','11','12','13','15','16','17','18','23',
    '25','26','27','29','30','31','32','37','40') THEN 1 ELSE 0 END AS FL_CHIP_3G,
    CASE 
        WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('14','19','21','28',
        '34','35','36','38','39','41','42','43','44','45','46','47','48',
        '49','50','51','52','53','54','55','56','57','58','59','60','62','66','67','71') THEN 1
        WHEN Substring(A.NR_SRAL From 1 FOR 6) = '895510' AND Substring(A.NR_SRAL From 10 FOR 2) IN ('92','93','94','95','96','97','98','99') THEN 1	
    ELSE 0 END AS FL_CHIP_4G,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('61','63','64','69') THEN 1 ELSE 0 END AS FL_CHIP_5G,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('52','54','55','62','63','64') THEN 1 ELSE 0 END AS FL_ESIM
FROM TMP_DIM_SRAL_V1 A
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

In [ ]:
import pandas, teradatasql, os
from dotenv import load_dotenv

load_dotenv()

TERADATA_HOST = str(os.environ.get("TERADATA_HOST"))
TERADATA_USER = str(os.environ.get("TERADATA_USER"))
TERADATA_PASSWORD = str(os.environ.get("TERADATA_PASSWORD"))

base = pandas.read_excel(
    r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\Restabelecimento de Velocidade\10.2023-11.2024_Restabelecimento de Velocidade - Visão MCM.xlsx",
    sheet_name="08.2024",
    dtype={"IMSI": "str", "MSISDN": "str"},
)

base["MSISDN_fix"] = base["MSISDN"].str.zfill(20)
base["MSISDN_fix"] = base["MSISDN_fix"].str.slice(9)

# Exportar informações do TAP para segmentação
COLS_TO_EXPORT = ["IMSI", "MSISDN_fix"]
linhas = (
    base.groupby(by=["IMSI", "MSISDN_fix"])
    .agg({"MB": "sum"})
    .reset_index()
    .drop_duplicates(subset=["IMSI", "MSISDN_fix"], keep="last")[COLS_TO_EXPORT]
    .to_csv(
        r"c:\Users\a0153041\Desktop\linhas_para_segmentar.csv",
        sep=",",
        decimal=".",
        encoding="utf_8",
        header=False,
        index=False,
        mode="w",
    )
)

# Exportar Informações para Teradata
TERADATA_PASSWORD = "Janeiro25!"


def query_teradata(query, connection, cursor, verbose=True):
    try:
        sRequest = query
        cursor.execute(sRequest, ignoreErrors=3807)
        if verbose:
            # obtain the warnings and errors for transmitting the data to the database -- the acquisition phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + query
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + query
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = (
                "{fn teradata_nativesql}{fn teradata_logon_sequence_number}" + query
            )
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]
    finally:
        pass


def insert_csv_data(table_name, values: int, cursor, file, connection, verbose=True):
    insert_values = ""
    for i in range(values):
        insert_values += "?, "
    query = "INSERT INTO " + table_name + " (" + insert_values[:-2] + ")"
    print(query)

    try:
        connection.autocommit = False
        try:
            sInsert = "{fn teradata_read_csv(" + str(file) + ")}" + query
            cursor.execute(sInsert)

            # obtain the warnings and errors for transmitting the data to the database -- the acquisition phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = (
                "{fn teradata_nativesql}{fn teradata_logon_sequence_number}" + sInsert
            )
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            connection.commit()

            # obtain the warnings and errors for the apply phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

        finally:
            connection.autocommit = True
    finally:
        if verbose:
            sRequest = f"SELECT TOP 5 * FROM {table_name}"
            print(sRequest)
            cursor.execute(sRequest, ignoreErrors=3807)
            print([row for row in cursor.fetchall()])


def export_full_table_to_csv(table_name, file_path, cursor):
    try:
        sRequest = f"SELECT * FROM {table_name}"
        sFileName = f"{file_path}/dataPy.csv"
        print(f"Exporting table data to file {sFileName}")
        cursor.execute("{fn teradata_write_csv(" + sFileName + ")}" + sRequest)
    finally:
        print("File exporting was successful!")


file = "c:/Users/a0153041/Desktop/linhas_para_segmentar.csv"

with teradatasql.connect(
    host=TERADATA_HOST,
    user=TERADATA_USER,
    password=TERADATA_PASSWORD,
    database="P_VIEDB",
    logmech="LDAP",
) as connect:
    with connect.cursor() as cur:
        query_teradata(SET_QUERY_BAND, connect, cur)
        query_teradata(RMNG_OUT_MAI, connect, cur)

        insert_csv_data("RMNG_OUT_MAI", 2, cur, file, connect)

        query_teradata(TB_TESTE_LINHAS_ID_PLNO, connect, cur)
        query_teradata(TB_TESTE_LINHAS_COMPLETO, connect, cur)
        query_teradata(TB_TESTE_LINHAS_SRVC, connect, cur)
        query_teradata(TMP_DIM_TAC_V1, connect, cur)
        query_teradata(TMP_DIM_TAC_V2, connect, cur)
        query_teradata(TMP_SRAL_LNHA, connect, cur)
        query_teradata(TMP_DIM_SRAL_V1, connect, cur)
        query_teradata(TMP_PRQE_FINAL, connect, cur)

        export_full_table_to_csv("TMP_PRQE_FINAL", f"c:/Users/a0153041/Desktop", cur)

# Importar informações segmentadas
linhas_segmentadas = pandas.read_csv(
    f"c:/Users/a0153041/Desktop/dataPy.csv",
    sep=",",
    encoding="utf_8",
    encoding_errors="ignore",
    on_bad_lines="skip",
    dtype={"IMSI": str, "NR_TLFN": str},
)
linhas_segmentadas.drop_duplicates(subset="IMSI", keep="first", inplace=True)

final = base.merge(linhas_segmentadas, on="IMSI", how="left")

final.to_excel(
    r"c:\Users\a0153041\Desktop\Rest. Vel. Segmentado\2024_08.xlsx", index=False
)

['']
['']
['']
['']
['']
['']
INSERT INTO RMNG_OUT_MAI (?, ?)
['']
['']
['']
['']
['']
SELECT TOP 5 * FROM RMNG_OUT_MAI
[[724065203314847, 51980522122], [724102005651242, 19999274050], [724112692836450, 21999925556], [724112904557388, 21971676770], [724102110021938, 11964763092]]
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
Exporting table data to file c:/Users/a0153041/Desktop/dataPy.csv
File exporting was successful!


# Exportar Informações

In [ ]:
class bcolors:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

In [ ]:
def formatSheets(writer: pandas.ExcelWriter, sheet: str, lenght: int = 1048576):
    workbook = writer.book
    worksheet = writer.sheets[sheet]
    lenght = str(lenght)

    # Formatação
    header = workbook.add_format({"bg_color": "#7030A0", "font_color": "#FFFFFF"})
    numberFormat = workbook.add_format(
        {
            "num_format": "#,##0.00",
            "border": 1,
            "font_size": 9,
            "align": "center",
            "valign": "vcenter",
        }
    )
    numberFormatNoPrecision = workbook.add_format(
        {
            "num_format": "0",
            "border": 1,
            "font_size": 9,
            "align": "center",
            "valign": "vcenter",
        }
    )
    usd = workbook.add_format(
        {
            "num_format": "$ #,##0.00",
            "border": 1,
            "font_size": 9,
            "align": "center",
            "valign": "vcenter",
        }
    )
    textFormat = workbook.add_format(
        {
            "num_format": "@",
            "border": 1,
            "font_size": 9,
            "align": "center",
            "valign": "vcenter",
        }
    )
    dateFormat = workbook.add_format(
        {
            "num_format": "dd/mm/yyyy",
            "border": 1,
            "font_size": 9,
            "align": "center",
            "valign": "vcenter",
        }
    )

    if sheet.startswith("Tráfego"):
        # Células a serem formatadas
        worksheet.conditional_format(
            "A1:AD1", {"type": "cell", "criteria": ">=", "value": 0, "format": header}
        )
        worksheet.conditional_format(
            "A2:B" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": dateFormat},
        )
        worksheet.conditional_format(
            "C2:G" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": textFormat},
        )
        worksheet.conditional_format(
            "H2:J" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": numberFormat},
        )
        worksheet.conditional_format(
            "K2:P" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": usd},
        )
        worksheet.conditional_format(
            "Q2:Y" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": textFormat},
        )
        worksheet.conditional_format(
            "Z2:AD" + lenght,
            {
                "type": "cell",
                "criteria": ">=",
                "value": 0,
                "format": numberFormatNoPrecision,
            },
        )

    if sheet.startswith("Resumo"):
        worksheet.conditional_format(
            "A1:G1", {"type": "cell", "criteria": ">=", "value": 0, "format": header}
        )
        worksheet.conditional_format(
            "A2:B" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": textFormat},
        )
        worksheet.conditional_format(
            "C2:C" + lenght,
            {
                "type": "cell",
                "criteria": ">=",
                "value": 0,
                "format": numberFormatNoPrecision,
            },
        )
        worksheet.conditional_format(
            "D2:G" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": numberFormat},
        )

    if sheet.startswith("Diárias_B2C") or sheet.startswith("Diárias_B2B"):
        worksheet.conditional_format(
            "A1:L1", {"type": "cell", "criteria": ">=", "value": 0, "format": header}
        )
        worksheet.conditional_format(
            "A2:A" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": dateFormat},
        )
        worksheet.conditional_format(
            "B2:C" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": textFormat},
        )
        worksheet.conditional_format(
            "D2:I" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": numberFormat},
        )
        worksheet.conditional_format(
            "J2:K" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": usd},
        )
        worksheet.conditional_format(
            "L2:L" + lenght,
            {"type": "cell", "criteria": ">=", "value": 0, "format": textFormat},
        )

    worksheet.freeze_panes(1, 0)
    worksheet.set_zoom(90)
    worksheet.autofit()


def exportReports(dataframes: dict, exportTo: str):
    try:
        dataframes["Tráfego"].to_parquet(f"{exportTo}.parquet", index=False)
    except Exception as e:
        pass

    # initialze the excel writer
    with pandas.ExcelWriter(exportTo + ".xlsx", engine="xlsxwriter") as writer:
        ROW_LIMIT = 1048576  # Excel row limit

        # now loop thru and put each on a specific sheet
        for sheet, frame in dataframes.items():  # .use .items for python 3.X

            if len(frame) > ROW_LIMIT:
                chunks = (len(frame) // ROW_LIMIT) + 1
                print(f"{bcolors.OKCYAN}Chunks: {chunks}{bcolors.ENDC}")

                # saves each chunk to a different sheet
                for i, chunk in enumerate(numpy.array_split(frame, chunks)):
                    temp_sheet = f"{sheet}-Parte_{i+1}"  # name of the sheet
                    print(
                        f"{bcolors.OKCYAN}Exportando planilha: {temp_sheet}...{bcolors.ENDC}"
                    )

                    chunk.to_excel(writer, sheet_name=temp_sheet, index=False)
                    formatSheets(writer, temp_sheet, len(chunk) + 1)

                    print(
                        f"{bcolors.OKGREEN}Planilha {temp_sheet} exportada com sucesso!{bcolors.ENDC}"
                    )
            else:
                print(f"Exportando planilha: {sheet}...")
                frame.to_excel(writer, sheet_name=sheet, index=False)
                formatSheets(writer, sheet, len(frame) + 1)
                print(
                    f"{bcolors.OKGREEN}Planilha {sheet} exportada com sucesso!{bcolors.ENDC}"
                )

## Diárias

In [ ]:
# sinalizacao.read_parquet(str(BASE_SINALIZACAO)+'/final.parquet')
sinalizacao = sinalizacao.merge(final[["IMSI", "SEGMENTO_fix"]], on="IMSI", how="left")
sinalizacao.to_parquet(str(BASE_TAP) + "/trafego_final.parquet")


def updateFile(file: str, sheet_name: str, dataframe: pandas.DataFrame):
    # create excel file
    if os.path.isfile(file):  # if file already exists append to existing file

        workbook = openpyxl.load_workbook(file)  # load workbook if already exists
        sheet = workbook[sheet_name]  # declare the active sheet

        # append the dataframe results to the current excel file
        for row in dataframe_to_rows(dataframe, header=False, index=False):
            dic = {
                "a": str(
                    f"{str(row[0].day).zfill(2)}/{str(row[0].month).zfill(2)}/{row[0].year}"
                ),
                "b": row[1],
                "c": row[2],
                "d": "",
                "e": row[3],
                "f": row[4],
                "g": "",
                "h": "",
                "i": row[5],
                "j": row[6],
                "k": "",
                "l": "",
                "m": "",
                "n": "",
            }
            print(type(row[0]))
            row = list(dic.values())
            sheet.append(row)
        workbook.save(file)  # save workbook
        workbook.close()  # close workbook
    else:  # create the excel file if doesn't already exist
        with pd.ExcelWriter(path=file, engine=ENGINE) as writer:
            dataframe.to_excel(writer, index=False, sheet_name=sheet_name)


def export_diarias_base(dataframes: dict, folder: str) -> None:
    for file, frame in dataframes.items():
        diarias = (
            frame.groupby(by=["Date (Call) YYYYMMDD", "IMSI", "DS_PLNO"], observed=True)
            .agg(
                {
                    "Charged SMS": "sum",
                    "Charged Minutes": "sum",
                    "Charged MB": "sum",
                    "MTC": "sum",
                    "MOC": "sum",
                    "Voz": "sum",
                    "Settlement Gross Charge - USD": "sum",
                    "Custo_USD": "sum",
                }
            )
            .reset_index()
        )
        diarias = (
            pandas.concat(
                [
                    diarias,
                    sinalizacao.groupby(
                        by=["Date (Call) YYYYMMDD", "IMSI"], observed=True
                    )
                    .agg({"MAP_Error_Name": "count"})
                    .reset_index(),
                ]
            )
            .sort_values(
                by=["Date (Call) YYYYMMDD", "Settlement Gross Charge - USD"],
                ascending=True,
            )
            .reset_index()
        )

        conditions = [
            (diarias["Charged SMS"] > 3)
            | (diarias["Charged Minutes"] > 0)
            | (diarias["Charged MB"] > 1)
        ]
        values = [True]
        diarias["isDiaria"] = numpy.select(conditions, values, default=False)

        # Resumos para a planilha de Análise de Diárias
        resumo_diarias_voz = (
            diarias[diarias["Charged Minutes"] > 0]
            .groupby(by=["Date (Call) YYYYMMDD"])
            .agg({"IMSI": "nunique"})
            .reset_index()
        )
        resumo_diarias_voz.rename(columns={"IMSI": "Qtde_Diárias_Voz"}, inplace=True)

        resumo_diarias_dados = (
            diarias[diarias["Charged MB"] > 1]
            .groupby(by=["Date (Call) YYYYMMDD"])
            .agg({"IMSI": "nunique"})
            .reset_index()
        )
        resumo_diarias_dados.rename(
            columns={"IMSI": "Qtde_Diárias_Dados"}, inplace=True
        )

        resumo_trafego = (
            diarias[diarias["isDiaria"] == True]
            .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
            .agg({"IMSI": "nunique", "Charged Minutes": "sum", "Charged MB": "sum"})
            .reset_index()
        )
        resumo_trafego.rename(
            columns={"IMSI": "Qtde_Diárias_Registradas"}, inplace=True
        )

        resumo_sinalizacao = (
            sinalizacao[sinalizacao["SEGMENTO_fix"] == file]
            .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
            .agg({"IMSI": "nunique"})
            .reset_index()
        )
        resumo_trafego.rename(columns={"IMSI": "Qtde_Registros"}, inplace=True)

        # Atualizar planilha de análise de diárias
        att = (
            resumo_diarias_voz.merge(
                resumo_diarias_dados, on="Date (Call) YYYYMMDD", how="left"
            )
            .merge(resumo_sinalizacao, on="Date (Call) YYYYMMDD", how="left")
            .merge(resumo_trafego, on="Date (Call) YYYYMMDD", how="left")
        )
        att = att[
            [
                "Date (Call) YYYYMMDD",
                "IMSI",
                "Qtde_Diárias_Registradas",
                "Qtde_Diárias_Voz",
                "Charged Minutes",
                "Qtde_Diárias_Dados",
                "Charged MB",
            ]
        ]
        att.rename(columns={"IMSI": "Qtde_Registros"}, inplace=True)

        if file == "B2B":
            updateFile(
                ANALISE_DIARIAS_B2B, "Dados", att
            )  # Atualiza a análise de diárias
        if file == "B2C":
            updateFile(ANALISE_DIARIAS_B2C, "Dados", att)
            resumo_sinalizacao = (
                sinalizacao[sinalizacao["SEGMENTO_fix"] == "B2C_POS"]
                .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
                .agg({"IMSI": "nunique"})
                .reset_index()
            )

        exportReports(
            {"Resumo_Atualizacao": att},
            f"{folder}resumo_diarias/atualizacao_analise/{file}",
        )  # Cria arquivo de diárias
        print(f"{bcolors.OKGREEN}Análise {file} exportada com sucesso!{bcolors.ENDC}")

        # Resumo da Base de Diárias
        dataframes_to_export = {
            "Resumo_Diárias_Voz": resumo_diarias_voz,
            "Resumo_Dados": resumo_diarias_dados,
            "Resumo_Sinalização": resumo_sinalizacao,
            "Resumo_Trafego": resumo_trafego,
        }
        exportReports(dataframes_to_export, f"{folder}resumo_diarias/{file}")
        print(f"{bcolors.OKGREEN}Resumo {file} exportado com sucesso!{bcolors.ENDC}")

        cols = [
            "Date (Call) YYYYMMDD",
            "IMSI",
            "DS_PLNO",
            "Charged SMS",
            "Charged Minutes",
            "Charged MB",
            "MTC",
            "MOC",
            "Voz",
            "Settlement Gross Charge - USD",
            "Custo_USD",
            "isDiaria",
        ]
        for col in cols:
            if col == "IMSI" or col == "DS_PLNO":
                diarias[col] = diarias[col].astype("category")

        diarias = (
            diarias.groupby(
                by=["Date (Call) YYYYMMDD", "IMSI", "DS_PLNO", "isDiaria"],
                observed=True,
            )
            .agg(
                {
                    "Charged SMS": "sum",
                    "Charged Minutes": "sum",
                    "Charged MB": "sum",
                    "MTC": "sum",
                    "MOC": "sum",
                    "Voz": "sum",
                    "Settlement Gross Charge - USD": "sum",
                    "Custo_USD": "sum",
                }
            )
            .reset_index()
        )

        resumo = (
            diarias.groupby(by=["Date (Call) YYYYMMDD", "isDiaria"])
            .agg({"IMSI": "count"})
            .reset_index()
        )

        dataframes_to_export = {
            "Resumo_Diárias": resumo,
            f"Diárias_{file}": diarias[cols],
        }
        # exportReports(dataframes_to_export, f"{folder}diarias_{file}")
        print(f"{bcolors.OKGREEN}Segmento {file} exportado com sucesso!{bcolors.ENDC}")
    del dataframes


dataframes = {
    "B2B": final[final["SEGMENTO_fix"] == "B2B"],
    "B2C": final[final["SEGMENTO_fix"] == "B2C_POS"],
}
export_diarias_base(dataframes, "base_final/")
del dataframes

'\ndef updateFile(file:str, sheet_name:str, dataframe:pandas.DataFrame):\n    # create excel file\n    if os.path.isfile(file):  # if file already exists append to existing file\n        \n        workbook = openpyxl.load_workbook(file)  # load workbook if already exists\n        sheet = workbook[sheet_name]  # declare the active sheet \n\n        # append the dataframe results to the current excel file\n        for row in dataframe_to_rows(dataframe, header = False, index = False):\n            dic = { \'a\':str(f\'{str(row[0].day).zfill(2)}/{str(row[0].month).zfill(2)}/{row[0].year}\'), \n                    \'b\':row[1],\n                    \'c\':row[2],\n                    \'d\':\'\',\n                    \'e\':row[3],\n                    \'f\':row[4],\n                    \'g\':\'\',\n                    \'h\':\'\',\n                    \'i\':row[5],\n                    \'j\':row[6],\n                    \'k\':\'\',\n                    \'l\':\'\',\n                    \'m\':\

## Tráfego

In [ ]:
COLS_TO_EXPORT = [
    "Date (Call) YYYYMMDD",
    "TAP File (Current) Processing Date",
    "PMN (Settlement) TADIG Code",
    "PMN (Settlement) Country",
    "IMSI",
    "MSISDN",
    "Call Type",
    "Charged SMS",
    "Charged Minutes",
    "Charged MB",
    "Settlement Gross Charge - USD",
    "Tarifa_Dados",
    "Tarifa_SMS",
    "Tarifa_MOC",
    "Tarifa_MTC",
    "Custo_USD",
    "DS_CRCT_PLNO",
    "DS_PLNO",
    "SEGMENTO_fix",
    "FABRICANTE",
    "MODELO",
    "DS_SIST_OPRC",
    "DS_CLSF_GSMA",
    "TECNOLOGIA_APARELHO",
    "NR_SRAL",
    "FL_DUAL_CHIP",
    "FL_CHIP_3G",
    "FL_CHIP_4G",
    "FL_CHIP_5G",
    "FL_ESIM",
]

export = {
    "B2C_POS_v2": b2c[COLS_TO_EXPORT],
    "B2B_v2": b2b[COLS_TO_EXPORT],
    "B2C_PRE_v2": pre[COLS_TO_EXPORT],
    "B2C_CONTROLE_v2": controle[COLS_TO_EXPORT],
}

for segmento, dataframe in export.items():
    print(f"{bcolors.OKCYAN}Exportando segmento: {segmento}...{bcolors.ENDC}")
    resumo = (
        dataframe.groupby(by=["PMN (Settlement) Country", "Call Type"])
        .agg(
            {
                "IMSI": "nunique",
                "Charged SMS": "sum",
                "Charged Minutes": "sum",
                "Charged MB": "sum",
                "Custo_USD": "sum",
            }
        )
        .reset_index()
    )

    dataframes_to_export = {"Resumo": resumo, "Tráfego": dataframe}

    exportReports(dataframes_to_export, REPORTS_FOLDER + f"_{segmento}")
    print(f"{bcolors.GREEN}Segmento {segmento} exportado com sucesso!{bcolors.ENDC}")

# Relatórios

### Resumo do Mês

In [ ]:
final.groupby(by=["PMN (Settlement) Country", "Call Type"]).agg(
    {
        "IMSI": "nunique",
        "Charged SMS": "sum",
        "Charged Minutes": "sum",
        "Charged MB": "sum",
        "Custo_USD": "sum",
    }
).reset_index()

### Analise Diarias

In [ ]:
# import pandas
# Importar informações segmentadas
MES_TAP = "202501"

folder = "C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/02-LINHAS_SEGMENTADAS"
sinalizacao = f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/00 - Dados/Sinalizacao/{MES_TAP}/final.parquet"

sinalizacao = pandas.read_parquet(sinalizacao)

linhas_segmentadas = pandas.read_csv(
    folder + f"/{MES_TAP}/output_dw/dataPy.csv",
    sep=",",
    encoding="utf_8",
    encoding_errors="ignore",
    on_bad_lines="skip",
    dtype={"IMSI": str, "NR_TLFN": str},
)
linhas_segmentadas.drop_duplicates(subset="IMSI", keep="first", inplace=True)

sinalizacao = sinalizacao.merge(linhas_segmentadas, on="IMSI", how="left")

sinalizacao[
    (sinalizacao["SEGMENTO"].str.startswith("PJ") == True)
    & (sinalizacao["Country"] != "Brazil")
].groupby(by=["Date (Call) YYYYMMDD"], observed=True).agg(
    {"IMSI": "nunique"}
).reset_index()

KeyboardInterrupt: 

In [ ]:
import pandas

ARQUIVO = r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_06.parquet"
SEGMENTO = "B2C_POS"  # B2B

try:
    diarias = (
        final.groupby(
            by=["IMSI", "SEGMENTO_fix", "Date (Call) YYYYMMDD"], observed=True
        )
        .agg(
            {
                "Charged SMS": "sum",
                "Charged MB": "sum",
                "Charged Minutes": "sum",
                "MOC": "sum",
                "MTC": "sum",
            }
        )
        .reset_index()
    )
except Exception as e:
    final = pandas.read_parquet(
        ARQUIVO,
        columns=[
            "Date (Call) YYYYMMDD",
            "IMSI",
            "SEGMENTO_fix",
            "Charged SMS",
            "Charged MB",
            "Charged Minutes",
            "MOC",
            "MTC",
        ],
    )
    diarias = (
        final.groupby(
            by=["IMSI", "SEGMENTO_fix", "Date (Call) YYYYMMDD"], observed=True
        )
        .agg(
            {
                "Charged SMS": "sum",
                "Charged MB": "sum",
                "Charged Minutes": "sum",
                "MOC": "sum",
                "MTC": "sum",
            }
        )
        .reset_index()
    )

# Diárias Non-Silent
NON_SILENT = (
    diarias[
        (diarias["SEGMENTO_fix"] == SEGMENTO)
        & (
            (diarias["Charged MB"] >= 1)
            | (diarias["Charged SMS"] > 0)
            | (diarias["Charged Minutes"] > 0)
        )
    ]
    .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
    .agg({"IMSI": "nunique"})
    .reset_index()
)

REGISTROS = (
    diarias[(diarias["SEGMENTO_fix"] == SEGMENTO)]
    .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
    .agg({"IMSI": "nunique"})
    .reset_index()
)

DATA_USED = (
    diarias[(diarias["SEGMENTO_fix"] == SEGMENTO) & ((diarias["Charged MB"] >= 1))]
    .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
    .agg({"IMSI": "nunique"})
    .reset_index()
)

VOICE_USED = (
    diarias[
        (diarias["SEGMENTO_fix"] == SEGMENTO)
        & ((diarias["MOC"] > 0) | (diarias["MTC"] > 0))
    ]
    .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
    .agg({"IMSI": "nunique"})
    .reset_index()
)

TOTAL_DATA_USED = (
    diarias[(diarias["SEGMENTO_fix"] == SEGMENTO)]
    .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
    .agg({"Charged MB": "sum"})
    .reset_index()
)
TOTAL_DATA_USED["Charged MB rounded"] = TOTAL_DATA_USED["Charged MB"].round(2)

TOTAL_VOICE_USED = (
    diarias[(diarias["SEGMENTO_fix"] == SEGMENTO)]
    .groupby(by=["Date (Call) YYYYMMDD"], observed=True)
    .agg({"MOC": "sum", "MTC": "sum"})
    .reset_index()
)
TOTAL_VOICE_USED["TOTAL"] = (TOTAL_VOICE_USED["MOC"] + TOTAL_VOICE_USED["MTC"]).round(2)

In [ ]:
NON_SILENT

,Date (Call) YYYYMMDD,IMSI
0,20250601,58040
1,20250602,59869
2,20250603,60175
3,20250604,60646
4,20250605,61348
5,20250606,62862
6,20250607,62549
7,20250608,60182
8,20250609,62088
9,20250610,62526


In [ ]:
REGISTROS

,Date (Call) YYYYMMDD,IMSI
0,20250601,86147
1,20250602,87637
2,20250603,87833
3,20250604,88223
4,20250605,89291
5,20250606,90625
6,20250607,91324
7,20250608,88616
8,20250609,90169
9,20250610,90554


In [ ]:
DATA_USED

,Date (Call) YYYYMMDD,IMSI
0,20250601,57400
1,20250602,58379
2,20250603,58666
3,20250604,59089
4,20250605,59786
5,20250606,61345
6,20250607,61533
7,20250608,59586
8,20250609,60499
9,20250610,61009


In [ ]:
VOICE_USED

,Date (Call) YYYYMMDD,IMSI
0,20250601,2721
1,20250602,6437
2,20250603,6825
3,20250604,6972
4,20250605,7092
5,20250606,7313
6,20250607,4474
7,20250608,2853
8,20250609,7216
9,20250610,7176


In [ ]:
TOTAL_DATA_USED

,Date (Call) YYYYMMDD,Charged MB,Charged MB rounded
0,20250601,3.383983e+07,3.383983e+07
1,20250602,3.106109e+07,3.106109e+07
2,20250603,3.077541e+07,3.077541e+07
3,20250604,3.095298e+07,3.095298e+07
4,20250605,3.176806e+07,3.176806e+07
5,20250606,3.264919e+07,3.264919e+07
6,20250607,3.402258e+07,3.402258e+07
7,20250608,3.469626e+07,3.469626e+07
8,20250609,3.226198e+07,3.226198e+07
9,20250610,3.133014e+07,3.133014e+07


In [ ]:
TOTAL_VOICE_USED

,Date (Call) YYYYMMDD,MOC,MTC,TOTAL
0,20250601,9994.533346,2235.283329,12229.82
1,20250602,16010.433359,5417.250006,21427.68
2,20250603,15671.000003,5992.616683,21663.62
3,20250604,15305.833315,6290.316671,21596.15
4,20250605,14687.300020,5620.933337,20308.23
5,20250606,15020.566664,6223.566672,21244.13
6,20250607,10775.700006,3551.216672,14326.92
7,20250608,9970.966670,2341.483328,12312.45
8,20250609,15219.733333,6485.333336,21705.07
9,20250610,15291.550005,6661.266683,21952.82


In [ ]:
B2C = diarias[diarias["SEGMENTO_fix"] == "B2C_POS"]["IMSI"].nunique()
B2C_PRE_CTRL = diarias[
    (diarias["SEGMENTO_fix"] == "B2C_PRE") | (diarias["SEGMENTO_fix"] == "B2C_CONTROLE")
]["IMSI"].nunique()
B2B = diarias[diarias["SEGMENTO_fix"] == "B2B"]["IMSI"].nunique()

NON_SILENT_B2C_POS = diarias[
    (
        (diarias["SEGMENTO_fix"] == "B2C_POS")
        & (
            (diarias["Charged MB"] >= 1)
            | (diarias["Charged SMS"] > 0)
            | (diarias["Charged Minutes"] > 0)
        )
    )
]["IMSI"].nunique()

NON_SILENT_B2C_PRE_CTRL = diarias[
    (
        (
            (diarias["SEGMENTO_fix"] == "B2C_PRE")
            | (diarias["SEGMENTO_fix"] == "B2C_CONTROLE")
        )
        & (
            (diarias["Charged MB"] >= 1)
            | (diarias["Charged SMS"] > 0)
            | (diarias["Charged Minutes"] > 0)
        )
    )
]["IMSI"].nunique()

NON_SILENT_B2B = diarias[
    (
        (diarias["SEGMENTO_fix"] == "B2B")
        & (
            (diarias["Charged MB"] >= 1)
            | (diarias["Charged SMS"] > 0)
            | (diarias["Charged Minutes"] > 0)
        )
    )
]["IMSI"].nunique()

print(
    f"B2B Total: {B2B}\tNON-SILENT: {NON_SILENT_B2B}\t% Non-Silent: {NON_SILENT_B2B/B2B*100}%"
)
print(
    f"B2C Pós Total: {B2C}\tNON-SILENT: {NON_SILENT_B2C_POS}\t% Non-Silent: {NON_SILENT_B2C_POS/B2C*100}%"
)
print(
    f"B2C Pré e Controle Total: {B2C_PRE_CTRL}\tNON-SILENT: {NON_SILENT_B2C_PRE_CTRL}\tNon-Silent: {NON_SILENT_B2C_PRE_CTRL/B2C_PRE_CTRL*100}%"
)

B2B Total: 79188	NON-SILENT: 63713	% Non-Silent: 80.45789766126181%
B2C Pós Total: 280335	NON-SILENT: 222974	% Non-Silent: 79.53840940303566%
B2C Pré e Controle Total: 426574	NON-SILENT: 126408	Non-Silent: 29.633310984729494%


### Linhas com Diárias Ativas

In [ ]:
SEGMENTO = "B2B"  # 'B2C_POS', 'B2C_CONTROLE', 'B2C_PRE', 'B2B'

report = (
    final[final["SEGMENTO_fix"] == SEGMENTO]
    .groupby(by=["Date (Call) YYYYMMDD", "NR_TLFN", "DS_PLNO"], observed=True)
    .agg(
        {
            "Settlement Charge - USD": "sum",
            "Charged MB": "sum",
            "Charged SMS": "sum",
            "MOC": "sum",
            "MTC": "sum",
        }
    )
    .reset_index()
)

report[
    (report["Charged MB"] >= 1)
    | (report["MOC"] > 0)
    | (report["MTC"] > 0)
    | (report["Charged SMS"] > 0)
].groupby(by=["NR_TLFN", "DS_PLNO"], observed=True).agg(
    {"Settlement Charge - USD": "sum"}
).reset_index()[
    ["NR_TLFN", "DS_PLNO"]
].to_excel(
    "B2B_202402.xlsx", index=False
)

### Ceconi IMSIs Únicos Ano

In [ ]:
import pandas


base = pandas.read_parquet(
    r"c:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\202412\final_segmentado.parquet"
)


base[base["SEGMENTO_fix"] == "B2B"].to_csv(
    "202412_B2B.csv.gz", sep=";", decimal=",", index=False, encoding="utf_16"
)

### Check Informações

In [ ]:
import pandas


MES_TAP = "202501"


base = pandas.read_parquet(
    f"c:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/00 - Dados/TAP/{MES_TAP}/final_segmentado.parquet",
    columns=["Date (Call) YYYYMMDD"],
)


base["Date (Call) YYYYMMDD"].unique().tolist()

[Timestamp('1970-01-01 00:00:00.020250101'),
 Timestamp('1970-01-01 00:00:00.020250102'),
 Timestamp('1970-01-01 00:00:00.020250103'),
 Timestamp('1970-01-01 00:00:00.020250104'),
 Timestamp('1970-01-01 00:00:00.020250105'),
 Timestamp('1970-01-01 00:00:00.020250106'),
 Timestamp('1970-01-01 00:00:00.020250107'),
 Timestamp('1970-01-01 00:00:00.020250108'),
 Timestamp('1970-01-01 00:00:00.020250109'),
 Timestamp('1970-01-01 00:00:00.020250110'),
 Timestamp('1970-01-01 00:00:00.020250111'),
 Timestamp('1970-01-01 00:00:00.020250112'),
 Timestamp('1970-01-01 00:00:00.020250113'),
 Timestamp('1970-01-01 00:00:00.020250114'),
 Timestamp('1970-01-01 00:00:00.020250115'),
 Timestamp('1970-01-01 00:00:00.020250116'),
 Timestamp('1970-01-01 00:00:00.020250117'),
 Timestamp('1970-01-01 00:00:00.020250118'),
 Timestamp('1970-01-01 00:00:00.020250119'),
 Timestamp('1970-01-01 00:00:00.020250120'),
 Timestamp('1970-01-01 00:00:00.020250121'),
 Timestamp('1970-01-01 00:00:00.020250122'),
 Timestamp

In [ ]:
import pandas

base = pandas.read_parquet(
    r"c:\Users\a0153041\OneDrive - Telefonica\03 - Relatórios\04 - Tráfego\00 - Trafego_Diario_Internacional\00 - Dados\TAP\202506\202506_NACIONAL.parquet"
)
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25637904 entries, 0 to 25637903
Data columns (total 38 columns):
 #   Column                                     Dtype   
---  ------                                     -----   
 0   TAP File (Current) Name                    object  
 1   TAP File (Current) Processing Date         object  
 2   Year-Month (Invoicing TAP)                 object  
 3   Date (Call) YYYYMMDD                       int64   
 4   PMN (Settlement) Country                   category
 5   PMN (Settlement) TADIG Code                category
 6   Call Type                                  category
 7   IMSI                                       category
 8   MSISDN                                     float64 
 9   Device TAC Code                            float64 
 10  Number of Calls (incl. combined partials)  float32 
 11  Charged SMS                                float32 
 12  Charged Minutes                            float32 
 13  Charged MB               

In [ ]:
base.groupby(by=["SEGMENTO"], observed=True).agg({"IMSI": "nunique"})

,IMSI
SEGMENTO,
NÃO INFORMADO,30
PF_CONTROLE,238785
PF_POS_FWA,7
PF_POS_PLACA,526
PF_POS_PURO,323872
PF_PRE_PLACA,28
PF_PRE_PURO,196250
PJ_CONTROLE,915
PJ_M2M,7603


### B2B - Cadu

In [2]:
import pandas

paths = [
    r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_11.parquet"
]

folder = "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\Compartilhado com Segmentos\\B2B\\"

for file in paths:
    print("File:", file)
    base = pandas.read_parquet(file, filters=[("SEGMENTO_fix", "==", "B2B")])
    print("Saving...")
    base.to_csv(
        folder + "b2b_" + file[-15:-8] + ".csv.gz",
        index=False,
        encoding="utf_16",
        sep=";",
        decimal=",",
    )
    print("Saved in ", (folder + "b2b_" + file[-15:-8] + ".csv.gz"))

File: c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_11.parquet
Saving...
Saved in  C:\Users\a0153041\Telefonica\Roaming - General\Compartilhado com Segmentos\B2B\b2b_2025_11.csv.gz


In [1]:
import pandas

pandas.read_csv(
    r"c:\Users\a0153041\Telefonica\Roaming - General\Compartilhado com Segmentos\B2B\b2b_2025_04_v2.csv.gz",
    encoding="utf_16",
    sep=";",
    decimal=",",
).head(20)

,TAP File (Current) Name,TAP File (Current) Processing Date,Year-Month (Invoicing TAP),Date (Call) YYYYMMDD,PMN (Settlement) Country,PMN (Settlement) TADIG Code,Call Type,IMSI,MSISDN,Device TAC Code,...,MOC_y,MTC_x,MTC_y,Voz,CN,MOC (Total),MTC (Total),GPRS (Total),SMS-MO (Total),Total
0,CDABWSEBRATC08774,1/4/2025,2025-04,20250401,Aruba,ABWSE,GPRS,724234291060210,5.531988e+10,NaN,...,0.15,0.0,0.0,0.000000,31,0.0000,0.0,0.410197,NaN,NaN
1,CDABWSEBRATC08775,2/4/2025,2025-04,20250401,Aruba,ABWSE,GPRS,724234291060210,5.531988e+10,86346604.0,...,0.15,0.0,0.0,0.000000,31,0.0000,0.0,0.116840,NaN,NaN
2,CDABWSEBRATC08775,2/4/2025,2025-04,20250401,Aruba,ABWSE,GPRS,724234291060210,5.531988e+10,NaN,...,0.15,0.0,0.0,0.000000,31,0.0000,0.0,1.367324,NaN,NaN
3,CDABWSEBRATC08775,2/4/2025,2025-04,20250401,Aruba,ABWSE,MTC,724234291060210,5.531988e+12,86346604.0,...,0.15,1.0,0.0,1.000000,31,0.0000,0.0,NaN,NaN,NaN
4,CDABWSEBRATC08776,2/4/2025,2025-04,20250401,Aruba,ABWSE,GPRS,724234291060210,5.531988e+10,NaN,...,0.15,0.0,0.0,0.000000,31,0.0000,0.0,0.068366,NaN,NaN
5,CDABWSEBRATC08776,2/4/2025,2025-04,20250402,Aruba,ABWSE,GPRS,724234291060210,5.531988e+10,86346604.0,...,0.15,0.0,0.0,0.000000,31,0.0000,0.0,0.001039,NaN,NaN
6,CDABWSEBRATC08776,2/4/2025,2025-04,20250402,Aruba,ABWSE,GPRS,724234291060210,5.531988e+10,NaN,...,0.15,0.0,0.0,0.000000,31,0.0000,0.0,0.136739,NaN,NaN
7,CDABWSEBRATC08777,3/4/2025,2025-04,20250402,Aruba,ABWSE,GPRS,724234291060210,5.531988e+10,86346604.0,...,0.15,0.0,0.0,0.000000,31,0.0000,0.0,0.204750,NaN,NaN
8,CDABWSEBRATC08777,3/4/2025,2025-04,20250402,Aruba,ABWSE,GPRS,724234291060210,5.531988e+10,NaN,...,0.15,0.0,0.0,0.000000,31,0.0000,0.0,1.914247,NaN,NaN
9,CDABWSEBRATC08777,3/4/2025,2025-04,20250402,Aruba,ABWSE,MOC,724234291060210,5.531988e+12,86346604.0,...,0.15,0.0,0.0,0.716667,31,0.1075,0.0,NaN,NaN,NaN


In [1]:
import pandas

base = pandas.read_parquet(
    r"c:\Users\a0153041\OneDrive - Telefonica\roaming-m2m\INBOUND\2025_08-INBOUND_CLARO.parquet"
)
base.head(5)

,TAP File (Current) Processing Date,Date (Call) YYYYMMDD,PMN (Settlement) TADIG Code,Call Type,IMSI,MSISDN,M2M Flag,APN Network,Device TAC Code,Number of Calls (incl. combined partials),Charged SMS,Charged Minutes,Charged MB,Settlement Gross Charge - USD
0,2025-08-02,2025-08-01,BRATA,GPRS,724050100001078,5511978773615,N,java.claro.com.br,86500106.0,0.0,0.0,0.0,0.001,0.0
1,2025-08-02,2025-08-01,BRATA,GPRS,724050100002660,5511994319607,N,java.claro.com.br,35560412.0,0.0,0.0,0.0,0.000,0.0
2,2025-08-02,2025-08-01,BRATA,GPRS,724050100007690,5511915473492,N,java.claro.com.br,35449213.0,0.0,0.0,0.0,0.001,0.0
3,2025-08-02,2025-08-01,BRATA,GPRS,724050100010587,5511970569891,N,java.claro.com.br,86424806.0,0.0,0.0,0.0,0.001,0.0
4,2025-08-02,2025-08-01,BRATA,GPRS,724050100013200,5511989128360,N,java.claro.com.br,86277607.0,0.0,0.0,0.0,0.002,0.0


# Nova Segmentação

In [ ]:
import pandas
import numpy
import pyarrow
import os
import calendar
import time
import pathlib
import teradatasql
import warnings
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

MES_TAP = "202507"

# PATHS
PATH = "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\DMVNO-C\\22 - Bases\\Sinalização\\silver\\seg_msisdn\\users_"
CDG_SAVE_PATH = "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\DMVNO-C\\22 - Bases\\CDG\\Nacional\\Resumos\\"
BASE_TAP = pathlib.Path("C:\\Users\\a0153041\\Downloads\\202507_NACIONAL")

BASE_FINAL = str(os.environ.get("BASE_FINAL")) + f"/{MES_TAP}"
REPORTS_FOLDER = str(os.environ.get("REPORTS")) + f"/{MES_TAP}"

# UTILS
TARIFFS = "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\Financeiro\\04 - Tarifas"
OUTPUT_FORMAT = ".csv"
TERADATA_HOST = str(os.environ.get("TERADATA_HOST"))  # "tdpvivo.redecorp.br"
TERADATA_USER = str(os.environ.get("TERADATA_USER"))  # "A0153041"
TERADATA_PASSWORD = str(os.environ.get("TERADATA_PASSWORD"))  # "Fevereiro29!"

# Supress Future Warnings
FUTURE_WARNINGS = str(os.environ.get("FUTURE_WARNINGS"))
warnings.simplefilter(action=FUTURE_WARNINGS, category=FutureWarning)
pandas.options.mode.copy_on_write = True


def load_signaling(path: str) -> pandas.DataFrame:
    return pandas.read_parquet(path)


def export_signaling_and_dch(
    sinalizacao: pandas.DataFrame, dch: pandas.DataFrame, year, month, SEG_FOLDER
) -> None:
    SEG_FOLDER  # f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/01-LINHAS_PARA_SEGMENTAR/{MES_TAP}"
    COLS_TO_EXPORT = ["IMSI", "MSISDN_fix"]

    try:
        # Exportar informações para segmentação
        linhas_sinalizacao = (
            sinalizacao[
                (sinalizacao.Date.dt.month == month)
                & (sinalizacao.Date.dt.year == year)
                & (
                    (sinalizacao["MSISDN_fix"] != "")
                    | (sinalizacao["MSISDN_fix"] != "0")
                )
            ]
            .groupby(by=["IMSI", "MSISDN_fix"], observed=True)
            .agg({"Country": "count"})
            .reset_index()
        )

        dch = dch[["IMSI", "MSISDN_fix", "Settlement Gross Charge - USD"]]
        dch["IMSI"] = dch["IMSI"].astype(str)
        dch["MSISDN_fix"] = dch["MSISDN_fix"].astype(str)
        dch = dch[dch["MSISDN_fix"] != "nan"]

        linhas_dch = (
            dch.groupby(by=["IMSI", "MSISDN_fix"])
            .agg({"Settlement Gross Charge - USD": "sum"})
            .reset_index()
            .drop_duplicates(subset=["IMSI", "MSISDN_fix"], keep="last")
        )

        linhas = pandas.concat(
            [linhas_sinalizacao[COLS_TO_EXPORT], linhas_dch[COLS_TO_EXPORT]]
        )

        linhas = linhas[COLS_TO_EXPORT].drop_duplicates(
            subset=["IMSI", "MSISDN_fix"], keep="last"
        )
        linhas.to_csv(
            SEG_FOLDER + "/linhas_para_segmentar.csv",
            sep=",",
            decimal=".",
            encoding="utf_8",
            header=False,
            index=False,
            mode="w",
        )
    except OSError:
        os.mkdir(SEG_FOLDER)
        linhas.to_csv(
            SEG_FOLDER + "/linhas_para_segmentar.csv",
            sep=",",
            decimal=".",
            encoding="utf_8",
            header=False,
            index=False,
            mode="w",
        )
    except Exception as e:
        print(e)
        return None
    finally:
        print("IMSIs Únicos Exportados: ", linhas["IMSI"].nunique())
        return None


def query_teradata(query, connection, cursor, verbose=True):
    try:
        sRequest = query
        cursor.execute(sRequest, ignoreErrors=3807)
        if verbose:
            # obtain the warnings and errors for transmitting the data to the database -- the acquisition phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + query
            cursor.execute(sRequest)
            # [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + query
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = (
                "{fn teradata_nativesql}{fn teradata_logon_sequence_number}" + query
            )
            cursor.execute(sRequest)
            # [print(row) for row in cursor.fetchall()]
    finally:
        pass


def insert_csv_data(table_name, values: int, cursor, file, connection, verbose=True):
    insert_values = ""
    for i in range(values):
        insert_values += "?, "
    query = "INSERT INTO " + table_name + " (" + insert_values[:-2] + ")"
    print(query)

    try:
        connection.autocommit = False
        try:
            sInsert = "{fn teradata_read_csv(" + file + ")}" + query
            cursor.execute(sInsert)

            # obtain the warnings and errors for transmitting the data to the database -- the acquisition phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + sInsert
            cursor.execute(sRequest)
            # [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + sInsert
            cursor.execute(sRequest)
            # [print(row) for row in cursor.fetchall()]

            sRequest = (
                "{fn teradata_nativesql}{fn teradata_logon_sequence_number}" + sInsert
            )
            cursor.execute(sRequest)
            # [print(row) for row in cursor.fetchall()]

            connection.commit()

            # obtain the warnings and errors for the apply phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + sInsert
            cursor.execute(sRequest)
            # [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + sInsert
            cursor.execute(sRequest)
            # [print(row) for row in cursor.fetchall()]

        finally:
            connection.autocommit = True
    finally:
        if verbose:
            sRequest = f"SELECT TOP 5 * FROM {table_name}"
            print(sRequest)
            cursor.execute(sRequest, ignoreErrors=3807)
            print([row for row in cursor.fetchall()])


def export_full_table_to_csv(table_name, file_path, cursor, year, month):
    try:
        sRequest = f"SELECT * FROM {table_name}"
        sFileName = f"{file_path}dw_{month}_{year}.csv"
        print(f"Exporting table data to file {sFileName}")
        # os.makedirs(
        #    f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/02-LINHAS_SEGMENTADAS/{MES_TAP}/output_dw/"
        # )
        cursor.execute("{fn teradata_write_csv(" + sFileName + ")}" + sRequest)
    finally:
        print("File exporting was successful!")


def adjustSegNames(dataframe) -> pandas.DataFrame:
    mapping = {
        "NÃO INFORMADO": "NÃO INFORMADO",
        "PF_CONTROLE": "B2C - CONTROLE",
        "PF_POS_FWA": "B2C - PÓS-PAGO",
        "PF_POS_PLACA": "B2C - PÓS-PAGO",
        "PF_POS_PURO": "B2C - PÓS-PAGO",
        "PF_PRE_PLACA": "B2C - PRÉ-PAGO",
        "PF_PRE_PURO": "B2C - PRÉ-PAGO",
        "PJ_CONTROLE": "B2B",
        "PJ_M2M": "B2B",
        "PJ_PLACA": "B2B",
        "PJ_PURO": "B2B",
    }

    dataframe["SEGMENTO_FINAL"] = dataframe["SEGMENTO_FINAL"].map(mapping)

    return dataframe


def seg_data(file, output_folder, year, month) -> pandas.DataFrame:
    START_DATE = f"{year}-{str(month).zfill(2)}-01"  # f'{MES_TAP[:4]}-{MES_TAP[4:]}-01'
    END_DATE = f"{year}-{str(month).zfill(2)}-{calendar.monthrange(year, month)[1]}"

    # Queries Teradata DW
    SET_QUERY_BAND = "SET QUERY_BAND = 'BLOCKCOMPRESSION=YES;' FOR SESSION;"
    RMNG_OUT_MAI = "CREATE VOLATILE TABLE RMNG_OUT_MAI (IMSI BIGINT, MSISDN BIGINT) PRIMARY INDEX (MSISDN) ON COMMIT PRESERVE ROWS;"

    TB_TESTE_LINHAS_ID_PLNO = f"""
    CREATE VOLATILE TABLE TB_TESTE_LINHAS_ID_PLNO AS (
    SELECT
        A.IMSI,
        A.MSISDN AS NR_TLFN,
        B.DS_PRDT AS SEGMENTO,
        B.ID_PSSA,
        B.ID_LNHA,
        B.ID_PLNO,
        B.DT_FOTO_LNHA,
        B.ID_CNTA,
        B.CD_CNTA,
        B.DT_PRMR_ATVC_LNHA,
        CASE
            WHEN B.DS_MODL_ORIG NOT LIKE ALL
                (
                    '%A%','%B%','%C%','%D%','%E%','%F%','%G%','%H%','%I%','%J%','%K%',
                    '%L%','%M%','%N%','%O%','%P%','%Q%','%R%','%S%','%T%','%U%','%V%',
                    '%W%','%X%','%Y%','%Z%','%#%','%/%','%@%','%\%','%-%','%.%','% %','%+%'
                )
                THEN Cast(B.DS_MODL_ORIG AS DECIMAL(8)) ELSE 0
        END AS NR_TAC
    FROM RMNG_OUT_MAI A
    LEFT JOIN P_VIEDB.VW_FAT_PRQE_LNHA_DSPT B
    ON A.MSISDN = B.NR_TLFN
    WHERE
        B.DT_FOTO_LNHA >= '{START_DATE}'
        AND B.DT_FOTO_LNHA <= '{END_DATE}'
        AND B.FL_PRQE_OFCL = 1
    QUALIFY ROW_NUMBER() OVER (PARTITION BY A.MSISDN ORDER BY B.DT_FOTO_LNHA DESC) = 1
    )
    WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;
    """

    TB_TESTE_LINHAS_COMPLETO = r"""
    CREATE VOLATILE TABLE TB_TESTE_LINHAS_COMPLETO AS(
    SELECT
        A.IMSI,
        A.NR_TLFN,
        A.ID_LNHA,
        B.DS_PLNO,
        A.SEGMENTO,
        A.ID_PSSA,
        B.DS_CRCT_PLNO,
        A.DT_FOTO_LNHA,
        A.NR_TAC
    FROM TB_TESTE_LINHAS_ID_PLNO A
    LEFT JOIN P_VIEDB.VW_DIM_PLNO B
    ON A.ID_PLNO = B.ID_PLNO
    )
    WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;
    """

    TB_TESTE_LINHAS_SRVC = f"""
    CREATE VOLATILE TABLE TB_TESTE_LINHAS_SRVC, NO LOG AS(
    SELECT 
        A.IMSI,
        A.NR_TLFN,
        A.ID_LNHA,
        A.DS_PLNO,
        A.SEGMENTO,
        A.ID_PSSA,
        A.DS_CRCT_PLNO,
        A.DT_FOTO_LNHA,
        B.ID_SRVC,
        B.DT_ATVC_SRVC,
        B.DT_DSTV_SRVC,
        A.NR_TAC
    FROM TB_TESTE_LINHAS_COMPLETO A
    LEFT JOIN P_VIEDB.VW_RLC_LNHA_SRVC B
    ON A.ID_LNHA = B.ID_LNHA
        AND B.ID_SRVC IN ('9693','555655','638298','947427','1014701','111946',
        '1036745','1036743','635896','448429','1036741','1967','1014791','118476',
        '1014755','1014719','953522','137962','947425','947429','1036737','558000',
        '1014675','45077','581827','605170','953578','602975','485618','146521',
        '120612','953354','118489','953466','953298','947439','123604','947431',
        '517949','1014773','123649','483073','118475','1036739','111947','981',
        '953186','445826','668060','1014737','134108','579498','953242','1036747',
        '45078','947441','947435','953410','1036751','947433','123605','1014685',
        '520347','137963','1036749','128758','405865','1015235','1015239','1015236',
        '1015241','1015238','1015242','1015237','1015240')
        AND B.DT_ATVC_SRVC <= '{END_DATE}' -- '2023-08-31'
        AND B.DT_DSTV_SRVC > '{END_DATE}' -- '2023-08-31'
    --GROUP BY 1,2,3,4,5
    QUALIFY ROW_NUMBER() OVER (PARTITION BY A.NR_TLFN ORDER BY B.ID_SRVC DESC) = 1
    )
    WITH DATA PRIMARY INDEX(ID_LNHA) ON COMMIT PRESERVE ROWS;
    """

    TMP_DIM_TAC_V1 = r"""
    CREATE VOLATILE TABLE TMP_DIM_TAC_V1, NO Log AS(
    SELECT
        A.*,
        B.DS_FBRC_ORIG AS FABRICANTE,
        B.DS_MODL_ORIG AS MODELO,
        B.DS_TCNL,
        B.ID_TCNL,
        B.FL_DUAL_CHIP,
        B.FL_SMPH AS FL_SMARTPHONE,
        B.FL_SMPH_LITE AS FL_SMART_LITE,
        B.DS_CRCT_ESPC,
        B.DS_SIST_OPRC,
        B.NM_MODL_GSMA,
        B.DS_SIST_OPRC_GSMA,
        B.DS_CLSF_GSMA,
        B.DS_CLSF_SMPH,
        CASE
            WHEN B.DS_TCNL LIKE '%Band 28%' THEN 1
            WHEN B.DS_TCNL LIKE '%BAND 28%' THEN 1
        ELSE 0 END AS FL_700Mhz
    FROM TB_TESTE_LINHAS_COMPLETO A
    LEFT JOIN P_VIEDB.VW_DIM_TAC B
    ON A.NR_TAC = B.NR_IMEI
    --WHERE ID_MES = 202408
    QUALIFY Row_Number() Over (PARTITION BY A.NR_TLFN ORDER BY ID_MES DESC) = 1
    )
    WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
    """

    TMP_DIM_TAC_V2 = r"""
    CREATE VOLATILE TABLE TMP_DIM_TAC_V2, NO Log AS(
    SELECT
        A.*,
        CASE
            WHEN B.NM_TCNL = 'GSM' THEN '2G'
            WHEN B.NM_TCNL = 'WCDMA' THEN '3G'
            WHEN B.NM_TCNL = 'LTE' THEN '4G'
        ELSE B.NM_TCNL END AS TECNOLOGIA_APARELHO
    FROM TMP_DIM_TAC_V1 A
    LEFT JOIN P_VIEDB.VW_DIM_TCNL B
    ON A.ID_TCNL = B.ID_TCNL
    --QUALIFY Row_Number() Over (PARTITION BY A.NR_TAC ORDER BY A.NR_TAC) = 1
    )
    WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
    """

    TMP_SRAL_LNHA = r"""
    CREATE VOLATILE TABLE TMP_SRAL_LNHA, NO Log AS(
    SELECT DISTINCT
        A.*,
        B.ID_TIPO_SRAL,
        B.NR_SRAL
    FROM TMP_DIM_TAC_V2 A
    LEFT JOIN P_VIEDB.VW_RLC_SRAL_LNHA B
    ON A.ID_LNHA = B.ID_LNHA
    WHERE (DT_VGNC_INCL <= (DATE - Extract(DAY From DATE)) AND DT_VGNC_FNAL > (DATE - Extract(DAY From DATE)))
    QUALIFY Row_Number() Over(PARTITION BY A.NR_TLFN ORDER BY A.NR_TLFN, B.DT_VGNC_INCL DESC ) = 1
    )
    WITH DATA PRIMARY INDEX(NR_TLFN) ON COMMIT PRESERVE ROWS;
    """

    TMP_DIM_SRAL_V1 = r"""
    CREATE VOLATILE TABLE TMP_DIM_SRAL_V1, NO Log AS(
    SELECT
        A.*,
        ID_LNHA_PRMR_ATVC
    FROM TMP_SRAL_LNHA A
    LEFT JOIN P_VIEDB.VW_DIM_SRAL B
    ON A.NR_SRAL = B.NR_SRAL
    QUALIFY Row_Number() Over(PARTITION BY A.NR_TLFN ORDER BY DT_PRMR_ATVC_SRAL DESC ) = 1
    )
    WITH DATA PRIMARY INDEX(NR_TLFN) ON COMMIT PRESERVE ROWS;
    """

    TMP_PRQE_FINAL = r"""
    CREATE VOLATILE TABLE TMP_PRQE_FINAL, NO Log AS(
    SELECT
        A.IMSI,
        A.NR_TLFN,
        A.NR_SRAL,
        A.DS_CRCT_PLNO,
        A.DS_PLNO,
        A.SEGMENTO,
        CAST(OREPLACE(A.FABRICANTE,',',' -') AS VARCHAR(256)) AS FABRICANTE,
        CAST(OREPLACE(A.MODELO,',',' -') AS VARCHAR(256)) AS MODELO,
        A.DS_SIST_OPRC,
        A.DS_CLSF_GSMA,
        A.TECNOLOGIA_APARELHO,
        A.FL_DUAL_CHIP,
        CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('01','02','03','04',
        '05','06','07','08','09','10','11','12','13','15','16','17','18','23',
        '25','26','27','29','30','31','32','37','40') THEN 1 ELSE 0 END AS FL_CHIP_3G,
        CASE 
            WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('14','19','21','28',
            '34','35','36','38','39','41','42','43','44','45','46','47','48',
            '49','50','51','52','53','54','55','56','57','58','59','60','62','66','67','71') THEN 1
            WHEN Substring(A.NR_SRAL From 1 FOR 6) = '895510' AND Substring(A.NR_SRAL From 10 FOR 2) IN ('92','93','94','95','96','97','98','99') THEN 1	
        ELSE 0 END AS FL_CHIP_4G,
        CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('61','63','64','69') THEN 1 ELSE 0 END AS FL_CHIP_5G,
        CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('52','54','55','62','63','64') THEN 1 ELSE 0 END AS FL_ESIM
    FROM TMP_DIM_SRAL_V1 A
    )
    WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
    """

    with teradatasql.connect(
        host=TERADATA_HOST,
        user=TERADATA_USER,
        password=TERADATA_PASSWORD,
        database="P_VIEDB",
        logmech="LDAP",
    ) as connect:
        with connect.cursor() as cur:
            query_teradata(SET_QUERY_BAND, connect, cur)
            query_teradata(RMNG_OUT_MAI, connect, cur)

            insert_csv_data("RMNG_OUT_MAI", 2, cur, file, connect)

            query_teradata(TB_TESTE_LINHAS_ID_PLNO, connect, cur)
            query_teradata(TB_TESTE_LINHAS_COMPLETO, connect, cur)
            query_teradata(TB_TESTE_LINHAS_SRVC, connect, cur)
            query_teradata(TMP_DIM_TAC_V1, connect, cur)
            query_teradata(TMP_DIM_TAC_V2, connect, cur)
            query_teradata(TMP_SRAL_LNHA, connect, cur)
            query_teradata(TMP_DIM_SRAL_V1, connect, cur)
            query_teradata(TMP_PRQE_FINAL, connect, cur)

            export_full_table_to_csv(
                "TMP_PRQE_FINAL", f"{output_folder}", cur, year, month
            )

    linhas_segmentadas = pandas.read_csv(
        output_folder + f"/dw_{month}_{year}.csv",
        sep=",",
        encoding="utf_8",
        encoding_errors="ignore",
        on_bad_lines="skip",
        dtype={"IMSI": str, "NR_TLFN": str},
    )
    linhas_segmentadas.drop_duplicates(subset="IMSI", keep="first", inplace=True)
    linhas_segmentadas.IMSI = pandas.to_numeric(
        linhas_segmentadas.IMSI, downcast="integer"
    )
    return linhas_segmentadas


def exec_segmentation(
    linhas: pandas.DataFrame, tap_report: pandas.DataFrame, year, month, SEG_FOLDER
) -> pandas.DataFrame:
    linhas_segmentadas = seg_data(
        f"{PATH}{month}_{year}{OUTPUT_FORMAT}",
        SEG_FOLDER,  # "C:/Users/a0153041/Telefonica/Roaming - General/DMVNO-C/22 - Bases/Sinalização/silver/seg_msisdn/",
        year,
        month,
    )[["IMSI", "SEGMENTO"]]

    print(
        f"{time.strftime("%d/%m/%y %H:%M [%z]")} - Segmentando {str(month).zfill(2)}/{year}..."
    )

    tap_report = tap_report.merge(linhas_segmentadas, how="left", on="IMSI")
    month_mask = (tap_report["Date (Call) YYYYMMDD"].dt.month == month) & (
        tap_report["Date (Call) YYYYMMDD"].dt.year == year
    )
    tap_report.loc[month_mask, "SEGMENTO_FINAL"] = tap_report["SEGMENTO"]
    tap_report.drop(columns=["SEGMENTO"], inplace=True)

    return tap_report


def generate_final_base(
    tap_report: pandas.DataFrame, linhas_segmentadas: pandas.DataFrame
) -> pandas.DataFrame:
    print("Cruzando informações...")

    for col in ["PMN (Settlement) Country", "PMN (Settlement) TADIG Code", "Call Type"]:
        tap_report[col] = tap_report[col].astype("category")

    for col in [
        "IMSI",
        "MSISDN",
        "Device TAC Code",
        "Number of Calls (incl. combined partials)",
        "Charged SMS",
        "Charged Minutes",
        "Charged MB",
        "Settlement Gross Charge - TAP Local Currency" "Settlement Charge - USD",
        "Settlement Tax - USD",
        "Settlement Gross Charge - USD",
        "Settlement Gross Charge - EUR",
    ]:
        try:
            tap_report[col] = pandas.to_numeric(tap_report[col], downcast="float")
        except KeyError:
            continue

    tap_report["MSISDN_fix"] = (
        tap_report["MSISDN_fix"].astype(str).str.replace(".0", "")
    )
    tap_report["MSISDN_fix"] = tap_report["MSISDN_fix"].str.replace("<NA>", "")
    tap_report["MSISDN_fix"] = tap_report["MSISDN_fix"].astype("category")

    tap_report["IMSI"] = tap_report["IMSI"].astype(str).str.replace(".0", "")
    tap_report["IMSI"] = tap_report["IMSI"].astype(str).str.replace("nan", "")
    tap_report["IMSI"] = tap_report["IMSI"].str.replace("<NA>", "")
    tap_report["IMSI"] = tap_report["IMSI"].astype("category")

    # Importar informações segmentadas
    linhas_segmentadas.drop_duplicates(subset="IMSI", keep="first", inplace=True)
    linhas_segmentadas["IMSI"] = linhas_segmentadas["IMSI"].astype(str)
    tap_report["IMSI"] = tap_report["IMSI"].astype(str)
    tap_report = tap_report.merge(linhas_segmentadas, on="IMSI", how="left")

    def add_values_to_dataframe(
        conditions: list, values: list, default=0
    ) -> pandas.Series:
        return numpy.select(conditions, values, default=default)

    print("Ajustando colunas...")
    conditions = [
        tap_report["SEGMENTO"].str.startswith("PF_POS") == True,
        tap_report["SEGMENTO"].str.startswith("PF_RESIDENCIAL") == True,
        tap_report["SEGMENTO"].str.startswith("PF_PRE") == True,
        tap_report["SEGMENTO"].str.startswith("PF_CONTROLE") == True,
        tap_report["SEGMENTO"].str.startswith("PJ") == True,
    ]
    values = ["B2C_POS", "B2C_POS", "B2C_PRE", "B2C_CONTROLE", "B2B"]
    tap_report["SEGMENTO_fix"] = add_values_to_dataframe(
        conditions, values, default=pandas.NA
    )

    # Atribuir os tipos corretos e excluir colunas desnecessárias
    cols = [
        "PMN (Settlement) Country",
        "PMN (Settlement) TADIG Code",
        "Call Type",
        "IMSI",
        "MSISDN",
        "MSISDN_fix",
        "NR_TLFN",
        "NR_SRAL",
        "DS_CRCT_PLNO",
        "DS_PLNO",
        "SEGMENTO",
        "FABRICANTE",
        "MODELO",
        "DS_SIST_OPRC",
        "DS_CLSF_GSMA",
        "TECNOLOGIA_APARELHO",
        "SEGMENTO_fix",
        "FL_DUAL_CHIP",
        "FL_CHIP_3G",
        "FL_CHIP_4G",
        "FL_CHIP_5G",
        "FL_ESIM",
    ]
    for col in cols:
        try:
            tap_report[col] = tap_report[col].astype("category")
        except KeyError:
            continue

    print("Criando colunas para análise de diárias...")
    # Para Diárias
    tap_report["MOC"] = tap_report[tap_report["Call Type"] == "MOC"]["Charged Minutes"]
    tap_report["MOC"] = tap_report["MOC"].astype(str).replace("nan", "0").astype(float)

    tap_report["MTC"] = tap_report[tap_report["Call Type"] == "MTC"]["Charged Minutes"]
    tap_report["MTC"] = tap_report["MTC"].astype(str).replace("nan", "0").astype(float)

    tap_report["Voz"] = tap_report["MOC"] + tap_report["MTC"]

    dados_utilizados = (
        tap_report.groupby(by=["SEGMENTO_fix"])
        .agg({"Charged MB": "sum", "IMSI": "nunique"})
        .reset_index()
    )
    dados_utilizados["Charged TB"] = dados_utilizados["Charged MB"] / 1024**2
    print(dados_utilizados)
    print(
        "Não Identificado: ",
        tap_report[tap_report["SEGMENTO_fix"].isna()]["IMSI"].nunique(),
        " IMSIs",
    )
    print(
        "Não Identificado: ",
        tap_report[tap_report["SEGMENTO_fix"].isna()]["Charged MB"].sum() / 1024**2,
        " TB",
    )

    tap_report.to_parquet(str(BASE_TAP) + f"/{MES_TAP}.parquet")
    return tap_report


def generate_cdg_report(
    type: Literal["NACIONAL", "INTERNACIONAL"],
    tap_report: pandas.DataFrame,
    cdg_path: str,
    tariffs: pandas.DataFrame,
) -> None:
    # cdg_path =    "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\DMVNO-C\\22 - Bases\\CDG\\Nacional\\Resumos\\"
    # tariffs  =    "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\Financeiro\\04 - Tarifas"
    if type == "NACIONAL":
        cdg_report = (
            tap_report.groupby(
                by=[
                    "PMN (Settlement) TADIG Code",
                    "PMN (Settlement) Country",
                    "Date (Call) YYYYMMDD",
                    "Call Type",
                    "IMSI",
                    "NR_TLFN",
                    "SEGMENTO",
                ],
                observed=True,
            )
            .agg(
                {
                    "Charged SMS": "sum",
                    "Charged MB": "sum",
                    "MOC": "sum",
                    "MTC": "sum",
                    "Settlement Gross Charge - TAP Local Currency": "sum",
                }
            )
            .reset_index()
        )

        cdg_report["CN"] = cdg_report["NR_TLFN"].astype(str).str[0:2]

        cdg_report["TARIFA_GPRS"] = (
            cdg_report["Settlement Gross Charge - TAP Local Currency"]
            / cdg_report["Charged MB"]
        )
        cdg_report["TARIFA_MOC"] = (
            cdg_report["Settlement Gross Charge - TAP Local Currency"]
            / cdg_report["MOC"]
        )
        cdg_report["TARIFA_MTC"] = (
            cdg_report["Settlement Gross Charge - TAP Local Currency"]
            / cdg_report["MTC"]
        )
        cdg_report["TARIFA_SMS"] = (
            cdg_report["Settlement Gross Charge - TAP Local Currency"]
            / cdg_report["Charged SMS"]
        )

        cdg_report["MOC (Total)"] = cdg_report["TARIFA_MOC"] * cdg_report["MOC"]
        cdg_report["MTC (Total)"] = cdg_report["TARIFA_MTC"] * cdg_report["MTC"]
        cdg_report["GPRS (Total)"] = (
            cdg_report["Charged MB"] * cdg_report["TARIFA_GPRS"]
        )
        cdg_report["SMS-MO (Total)"] = (
            cdg_report["Charged SMS"] * cdg_report["TARIFA_SMS"]
        )
        cdg_report["Total"] = (
            cdg_report["MOC (Total)"]
            + cdg_report["MTC (Total)"]
            + cdg_report["GPRS (Total)"]
            + cdg_report["SMS-MO (Total)"]
        )

        cdg_report = (
            cdg_report.groupby(
                by=["CN", "SEGMENTO", "Call Type"], observed=True, dropna=False
            )
            .agg(
                {
                    "Charged MB": "sum",
                    "Charged SMS": "sum",
                    "MOC": "sum",
                    "MTC": "sum",
                    "Settlement Gross Charge - TAP Local Currency": "sum",
                }
            )
            .reset_index()
            .pivot(
                columns="Call Type",
                index=["CN", "SEGMENTO"],
                values="Settlement Gross Charge - TAP Local Currency",
            )
            .reset_index()
        )
        cdg_report.to_excel(
            f"{cdg_path}{MES_TAP}_Nacional.xlsx",
            index=False,
        )
        return None
    elif type == "INTERNACIONAL":

        cdg = (
            tap_report.groupby(
                by=[
                    "PMN (Settlement) TADIG Code",
                    "PMN (Settlement) Country",
                    "Date (Call) YYYYMMDD",
                    "Call Type",
                    "IMSI",
                    "NR_TLFN",
                    "SEGMENTO",
                ],
                observed=True,
            )
            .agg(
                {"Charged SMS": "sum", "Charged MB": "sum", "MOC": "sum", "MTC": "sum"}
            )
            .reset_index()
        )

        cdg = cdg.merge(tariffs, how="left", on="PMN (Settlement) TADIG Code")

        cdg["PMN (Settlement) TADIG Code"] = cdg["PMN (Settlement) TADIG Code"].astype(
            "category"
        )
        cdg["ALPHA_3"] = cdg["ALPHA_3"].astype("category")
        cdg["short_name_ptbr"] = cdg["short_name_ptbr"].astype("category")
        cdg["region"] = cdg["region"].astype("category")
        cdg["sub-region"] = cdg["sub-region"].astype("category")
        cdg["MOEDA"] = cdg["MOEDA"].astype("category")

        cdg["Date (Call) YYYYMMDD"] = pandas.to_numeric(
            cdg["Date (Call) YYYYMMDD"], downcast="unsigned"
        )

        cdg["CN"] = cdg["NR_TLFN"].astype(str).str[0:2]
        cdg["CN"] = cdg["CN"].astype("category")

        cdg["MOC (Total)"] = cdg["ACORDO_MOC"] * cdg["MOC"]
        cdg["MTC (Total)"] = cdg["ACORDO_MTC"] * cdg["MTC"]
        cdg["GPRS (Total)"] = cdg["Charged MB"] * cdg["ACORDO_GPRS"]
        cdg["SMS-MO (Total)"] = cdg["Charged SMS"] * cdg["ACORDO_SMS_MO"]
        cdg["Total"] = (
            cdg["MOC (Total)"]
            + cdg["MTC (Total)"]
            + cdg["GPRS (Total)"]
            + cdg["SMS-MO (Total)"]
        )

        cols = [
            "PMN (Settlement) TADIG Code",
            "PMN (Settlement) Country",
            "Date (Call) YYYYMMDD",
            "Call Type",
            "IMSI",
            "MSISDN",
            "Charged SMS",
            "Charged Minutes",
            "Charged MB",
            "SEGMENTO" "MOC_x",
            "MTC_x",
            "CN",
            "MOC",
            "MTC",
            "SMS-MO",
            "GPRS",
            "MOC (Total)",
            "MTC (Total)",
            "GPRS (Total)",
            "SMS-MO (Total)",
            "Total",
        ]

        cdg.groupby(by=["CN", "SEGMENTO"], observed=True, dropna=False).agg(
            {
                "GPRS (Total)": "sum",
                "SMS-MO (Total)": "sum",
                "MOC (Total)": "sum",
                "MTC (Total)": "sum",
                "Total": "sum",
            }
        ).reset_index().to_excel(
            f"{cdg_path}{MES_TAP}_Internacional.xlsx",
            index=False,
        )
    else:
        print("[ERROR] TYPE NOT RECOGNIZED!")
        return None


def load_tap_data(BASE_TAP: pathlib.Path) -> pandas.DataFrame:
    paths = sorted(BASE_TAP.iterdir(), key=os.path.getmtime)
    bases = []
    print(paths)

    for file in paths:
        if (
            str(file).endswith("final.csv")
            or str(file).endswith(".parquet")
            or str(file).endswith(".zip")
            or str(file).endswith("final.csv.gz")
        ):
            continue

        print(f"Loading file: {file}")
        bases.append(
            pandas.read_csv(
                file,
                sep=";",
                decimal=",",
                encoding="utf_16",
                dtype={"IMSI": str, "MSISDN": str},
                low_memory=False,
                skiprows=0,
            )
        )

    tap = pandas.concat(bases)
    export_tap_original(tap, BASE_TAP, MES_TAP, suffix="INTERNACIONAL")
    return tap


def adjust_columns_tap(tap_report: pandas.DataFrame) -> pandas.DataFrame:

    NUMERIC_COLS = [
        "Device TAC Code",
        "Number of Calls (incl. combined partials)",
        "Charged SMS",
        "Charged Minutes",
        "Charged MB",
        "Settlement Gross Charge - TAP Local Currency",
        "Settlement Charge - USD",
        "Settlement Tax - USD",
        "Settlement Gross Charge - USD",
        "Settlement Gross Charge - EUR",
    ]
    CATEGORY_COLS = [
        "PMN (Settlement) Country",
        "PMN (Settlement) TADIG Code",
        "Call Type",
        "PMN (Settlement) Name",
        "Year-Month (Call)",
    ]

    tap_report = set_column_to_numeric(tap_report, NUMERIC_COLS, "float")
    for col in CATEGORY_COLS:
        tap_report[col] = tap_report[col].astype("category")

    tap_report.IMSI = tap_report.IMSI.astype(str).str.replace(".0", "")
    tap_report.IMSI = tap_report.IMSI.str.replace("<NA>", "")
    tap_report.IMSI = tap_report.IMSI.astype("category")

    tap_report["MSISDN_fix"] = tap_report.MSISDN.str.zfill(20)
    tap_report.MSISDN_fix = tap_report.MSISDN_fix.str.slice(9)
    tap_report.MSISDN_fix = tap_report.MSISDN_fix.astype(str).str.replace(".0", "")
    tap_report.MSISDN_fix = tap_report.MSISDN_fix.str.replace("<NA>", "")
    tap_report.MSISDN_fix = tap_report.MSISDN_fix.astype("category")

    tap["TAP File (Current) Processing Date"] = pandas.to_datetime(
        tap["TAP File (Current) Processing Date"], dayfirst=True
    )
    tap_report["Date (Call) YYYYMMDD"] = pandas.to_datetime(
        tap_report["Date (Call) YYYYMMDD"], format="%Y%m%d"
    )

    try:
        cols_to_drop = []
        for i, v in enumerate(tap_report.columns.tolist()):
            if v.__contains__("Unnamed"):
                cols_to_drop.append(v)
        tap_report.drop(columns=cols_to_drop, inplace=True)
    except KeyError:
        pass

    return tap_report


def export_tap_original(
    tap_report: pandas.DataFrame, BASE_TAP: pathlib.Path, MES_TAP: str, suffix: str = ""
) -> None:
    tap_report.to_csv(
        str(BASE_TAP) + f"/{MES_TAP}_{suffix}.csv.gz",
        sep=";",
        decimal=",",
        encoding="utf_16",
    )
    return None


def set_column_to_numeric(
    dataframe: pandas.DataFrame, cols: list[str], downcast: str = "float"
) -> pandas.DataFrame:
    for col in cols:
        try:
            dataframe[col] = pandas.to_numeric(dataframe[col], downcast=downcast)
        except KeyError:
            continue
        except ValueError:
            continue
    return dataframe


def load_tariffs(tariffs: str) -> pandas.DataFrame:
    paths = sorted(
        pathlib.Path(tariffs).iterdir(),
        key=os.path.getmtime,
    )

    iot_paths = []
    for i, v in enumerate(paths):
        if "TARIFAS_ATUAIS" in str(v):
            iot_paths.append(v)
    iot_path = sorted(iot_paths)[0]
    tarifas = pandas.read_excel(iot_path)
    tarifas.rename(columns={"PLMN": "PMN (Settlement) TADIG Code"}, inplace=True)
    cols = ["ACORDO_MOC", "ACORDO_MTC", "ACORDO_SMS_MO", "ACORDO_GPRS"]
    for col in cols:
        tarifas[tarifas[col] == "Gross"] = 0.0
        tarifas[col] = pandas.to_numeric(tarifas[col], downcast="float", errors="raise")
    return tarifas


def export_arco_b2b(dataframe: pandas.DataFrame, destination: str) -> None:

    try:
        dataframe = dataframe[dataframe["SEGMENTO_fix"] == "B2B"]
    except KeyError:
        dataframe = dataframe[dataframe["SEGMENTO_fix"] == "B2B"]
    finally:
        tarifas = load_tariffs(TARIFFS)

        final = base.merge(tarifas, how="left", on="PMN (Settlement) TADIG Code")

        final["CN"] = final["NR_TLFN"].astype(str).str[0:2]
        final["MOC (Total)"] = final["MOC"] * final["ACORDO_MOC"]
        final["MTC (Total)"] = final["MTC"] * final["ACORDO_MTC"]
        final["GPRS (Total)"] = final["Charged MB"] * final["ACORDO_GPRS"]
        final["SMS-MO (Total)"] = final["Charged SMS"] * final["ACORDO_SMS_MO"]
        final["Total"] = (
            final["MOC (Total)"]
            + final["MTC (Total)"]
            + final["GPRS (Total)"]
            + final["SMS-MO (Total)"]
        )

        print(f"Saving ARCO B2B report...")
        final[cols].to_csv(
            destination,  #  "c:\\Users\\a0153041\\Desktop\\2025_06.csv",
            decimal=",",
            sep=";",
            encoding="utf_16",
            index=False,
        )
        print(f"Saved!")

In [ ]:
def run_pipeline() -> None:

    load_dotenv()

    MES_TAP = "202507"

    # PATHS
    PATH = "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\DMVNO-C\\22 - Bases\\Sinalização\\silver\\seg_msisdn\\users_"
    CDG_SAVE_PATH = "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\DMVNO-C\\22 - Bases\\CDG\\Nacional\\Resumos\\"
    BASE_TAP = pathlib.Path("C:\\Users\\a0153041\\Downloads\\202507_NACIONAL")
    BASE_SINALIZACAO = pathlib.Path(str(os.environ.get("BASE_SINALIZACAO")))

    BASE_FINAL = str(os.environ.get("BASE_FINAL")) + f"/{MES_TAP}"
    REPORTS_FOLDER = str(os.environ.get("REPORTS")) + f"/{MES_TAP}"

    # UTILS
    TARIFFS = (
        "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\Financeiro\\04 - Tarifas"
    )
    OUTPUT_FORMAT = ".csv"
    TERADATA_HOST = str(os.environ.get("TERADATA_HOST"))  # "tdpvivo.redecorp.br"
    TERADATA_USER = str(os.environ.get("TERADATA_USER"))  # "A0153041"
    TERADATA_PASSWORD = str(os.environ.get("TERADATA_PASSWORD"))  # "Fevereiro29!"

    # Supress Future Warnings
    FUTURE_WARNINGS = str(os.environ.get("FUTURE_WARNINGS"))
    warnings.simplefilter(action=FUTURE_WARNINGS, category=FutureWarning)
    pandas.options.mode.copy_on_write = True

    tarifas = load_signaling(str(BASE_SINALIZACAO))

    return None

## Segmentação Nacional

In [2]:
import pandas
import numpy
import pyarrow
import os
import pathlib
import teradatasql
import warnings

# import openpyxl
# from openpyxl.utils.dataframe import dataframe_to_rows
from dotenv import load_dotenv

In [40]:
MES_TAP = "202507"

load_dotenv()
BASE_TAP = pathlib.Path("C:\\Users\\a0153041\\Downloads\\202507_NACIONAL")

BASE_FINAL = str(os.environ.get("BASE_FINAL")) + f"/{MES_TAP}"
REPORTS_FOLDER = str(os.environ.get("REPORTS")) + f"/{MES_TAP}"

FUTURE_WARNINGS = str(os.environ.get("FUTURE_WARNINGS"))

TERADATA_HOST = str(os.environ.get("TERADATA_HOST"))
TERADATA_USER = str(os.environ.get("TERADATA_USER"))
TERADATA_PASSWORD = str(os.environ.get("TERADATA_PASSWORD"))

# Supress Future Warnings
warnings.simplefilter(action=FUTURE_WARNINGS, category=FutureWarning)
pandas.options.mode.copy_on_write = True

In [ ]:
paths = sorted(BASE_TAP.iterdir(), key=os.path.getmtime)
bases = []
print(paths)

for file in paths:
    if (
        str(file).endswith("final.csv")
        or str(file).endswith(".parquet")
        or str(file).endswith(".zip")
        or str(file).endswith("final.csv.gz")
    ):
        continue

    print(f"Loading file: {file}")
    bases.append(
        pandas.read_csv(
            file,
            sep=";",
            decimal=",",
            encoding="utf_16",
            dtype={"IMSI": str, "MSISDN": str},
            low_memory=False,
            skiprows=0,
        )
    )

tap = pandas.concat(bases)
del bases

tap["MSISDN_fix"] = tap["MSISDN"].str.zfill(20)
tap["MSISDN_fix"] = tap["MSISDN_fix"].str.slice(9)

tap.to_csv(
    str(BASE_TAP) + f"/{MES_TAP}_NACIONAL.csv.gz",
    sep=";",
    decimal=",",
    encoding="utf_16",
)

for col in ["PMN (Settlement) Country", "PMN (Settlement) TADIG Code", "Call Type"]:
    tap[col] = tap[col].astype("category")

tap["Date (Call) YYYYMMDD"] = pandas.to_datetime(
    tap["Date (Call) YYYYMMDD"], format="%Y%m%d"
)

for col in [
    "IMSI",
    "MSISDN",
    "Device TAC Code",
    "Number of Calls (incl. combined partials)",
    "Charged SMS",
    "Charged Minutes",
    "Charged MB",
    "Settlement Gross Charge - TAP Local Currency",
    "Settlement Charge - USD",
    "Settlement Tax - USD",
    "Settlement Gross Charge - USD",
    "Settlement Gross Charge - EUR",
]:
    try:
        tap[col] = pandas.to_numeric(tap[col], downcast="float")
    except KeyError:
        continue

tap["MSISDN_fix"] = tap["MSISDN_fix"].astype(str).str.replace(".0", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].str.replace("<NA>", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].astype("category")

tap["IMSI"] = tap["IMSI"].astype(str).str.replace(".0", "")
tap["IMSI"] = tap["IMSI"].str.replace("<NA>", "")
tap["IMSI"] = tap["IMSI"].astype("category")

try:
    tap.drop(columns=["Unnamed: 0"], inplace=True)
except KeyError:
    pass

tap.info()

In [6]:
tap["Date (Call) YYYYMMDD"] = pandas.to_datetime(
    tap["Date (Call) YYYYMMDD"], format="%Y%m%d"
)

for col in [
    "IMSI",
    "MSISDN",
    "Device TAC Code",
    "Number of Calls (incl. combined partials)",
    "Charged SMS",
    "Charged Minutes",
    "Charged MB",
    "Settlement Gross Charge - TAP Local Currency",
    "Settlement Charge - USD",
    "Settlement Tax - USD",
    "Settlement Gross Charge - USD",
    "Settlement Gross Charge - EUR",
]:
    try:
        tap[col] = pandas.to_numeric(tap[col], downcast="float")
    except KeyError:
        continue

tap["MSISDN_fix"] = tap["MSISDN_fix"].astype(str).str.replace(".0", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].str.replace("<NA>", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].astype("category")

tap["IMSI"] = tap["IMSI"].astype(str).str.replace(".0", "")
tap["IMSI"] = tap["IMSI"].str.replace("<NA>", "")
tap["IMSI"] = tap["IMSI"].astype("category")

try:
    tap.drop(columns=["Unnamed: 0"], inplace=True)
except KeyError:
    pass

tap.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8830865 entries, 0 to 1529398
Data columns (total 15 columns):
 #   Column                                        Dtype         
---  ------                                        -----         
 0   Year-Month (Call)                             object        
 1   Date (Call) YYYYMMDD                          datetime64[ns]
 2   PMN (Settlement) Country                      category      
 3   PMN (Settlement) Name                         object        
 4   PMN (Settlement) TADIG Code                   category      
 5   IMSI                                          category      
 6   MSISDN                                        float64       
 7   Location Area Code                            float64       
 8   Cell ID                                       float64       
 9   Call Type                                     category      
 10  Charged SMS                                   float32       
 11  Charged Minutes              

In [7]:
COLS_TO_EXPORT = ["IMSI", "MSISDN_fix"]
try:
    temp = tap[["IMSI", "MSISDN_fix", "Settlement Gross Charge - USD"]]
    temp["IMSI"] = temp["IMSI"].astype(str)
    temp["MSISDN_fix"] = temp["MSISDN_fix"].astype(str)

    temp = temp[temp["MSISDN_fix"] != "nan"]

    linhas = (
        temp.groupby(by=["IMSI", "MSISDN_fix"])
        .agg({"Settlement Gross Charge - USD": "sum"})
        .reset_index()
        .drop_duplicates(subset=["IMSI", "MSISDN_fix"], keep="last")
    )
except KeyError:
    temp = tap[["IMSI", "MSISDN_fix", "Settlement Gross Charge - TAP Local Currency"]]
    temp["IMSI"] = temp["IMSI"].astype(str)
    temp["MSISDN_fix"] = temp["MSISDN_fix"].astype(str)

    temp = temp[temp["MSISDN_fix"] != "nan"]

    linhas = (
        temp.groupby(by=["IMSI", "MSISDN_fix"])
        .agg({"Settlement Gross Charge - TAP Local Currency": "sum"})
        .reset_index()
        .drop_duplicates(subset=["IMSI", "MSISDN_fix"], keep="last")
    )

linhas = linhas[linhas["MSISDN_fix"] != "00000000000"]

try:
    linhas[COLS_TO_EXPORT].to_csv(
        f"{str(BASE_TAP)}/linhas_para_segmentar.csv",
        sep=",",
        decimal=".",
        encoding="utf_8",
        header=False,
        index=False,
        mode="w",
    )
except OSError:
    os.mkdir(
        f"C:/Users/a0153041/OneDrive - Telefonica/03 - Relatórios/04 - Tráfego/00 - Trafego_Diario_Internacional/02 - DW/01-LINHAS_PARA_SEGMENTAR/{MES_TAP}"
    )
    linhas[COLS_TO_EXPORT].to_csv(
        f"{str(BASE_TAP)}/linhas_para_segmentar.csv",
        sep=",",
        decimal=".",
        encoding="utf_8",
        header=False,
        index=False,
        mode="w",
    )

In [ ]:
# Queries Teradata DW

SET_QUERY_BAND = "SET QUERY_BAND = 'BLOCKCOMPRESSION=YES;' FOR SESSION;"
RMNG_OUT_MAI = "CREATE VOLATILE TABLE RMNG_OUT_MAI (IMSI BIGINT, MSISDN BIGINT) PRIMARY INDEX (MSISDN) ON COMMIT PRESERVE ROWS;"

START_DATE = "2025-07-01"  # f'{MES_TAP[:4]}-{MES_TAP[4:]}-01'
END_DATE = "2025-07-31"  # calendar.monthrange(int(MES_TAP[:4]), int(MES_TAP[4:]))[1]

TB_TESTE_LINHAS_ID_PLNO = f"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_ID_PLNO AS (
SELECT
    A.IMSI,
    A.MSISDN AS NR_TLFN,
    B.DS_PRDT AS SEGMENTO,
    B.ID_PSSA,
    B.ID_LNHA,
    B.ID_PLNO,
    B.DT_FOTO_LNHA,
    B.ID_CNTA,
    B.CD_CNTA,
    B.DT_PRMR_ATVC_LNHA,
    CASE
        WHEN B.DS_MODL_ORIG NOT LIKE ALL
            (
                '%A%','%B%','%C%','%D%','%E%','%F%','%G%','%H%','%I%','%J%','%K%',
                '%L%','%M%','%N%','%O%','%P%','%Q%','%R%','%S%','%T%','%U%','%V%',
                '%W%','%X%','%Y%','%Z%','%#%','%/%','%@%','%\%','%-%','%.%','% %','%+%'
            )
            THEN Cast(B.DS_MODL_ORIG AS DECIMAL(8)) ELSE 0
    END AS NR_TAC
FROM RMNG_OUT_MAI A
LEFT JOIN P_VIEDB.VW_FAT_PRQE_LNHA_DSPT B
ON A.MSISDN = B.NR_TLFN
WHERE
    B.DT_FOTO_LNHA >= '{START_DATE}'
    AND B.DT_FOTO_LNHA <= '{END_DATE}'
    AND B.FL_PRQE_OFCL = 1
QUALIFY ROW_NUMBER() OVER (PARTITION BY A.MSISDN ORDER BY B.DT_FOTO_LNHA DESC) = 1
)
WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TB_TESTE_LINHAS_COMPLETO = r"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_COMPLETO AS(
SELECT
    A.IMSI,
    A.NR_TLFN,
    A.ID_LNHA,
    B.DS_PLNO,
    A.SEGMENTO,
    A.ID_PSSA,
    B.DS_CRCT_PLNO,
    A.DT_FOTO_LNHA,
    A.NR_TAC
FROM TB_TESTE_LINHAS_ID_PLNO A
LEFT JOIN P_VIEDB.VW_DIM_PLNO B
ON A.ID_PLNO = B.ID_PLNO
)
WITH DATA PRIMARY INDEX (ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TB_TESTE_LINHAS_SRVC = f"""
CREATE VOLATILE TABLE TB_TESTE_LINHAS_SRVC, NO LOG AS(
SELECT 
    A.IMSI,
    A.NR_TLFN,
    A.ID_LNHA,
    A.DS_PLNO,
    A.SEGMENTO,
    A.ID_PSSA,
    A.DS_CRCT_PLNO,
    A.DT_FOTO_LNHA,
    B.ID_SRVC,
    B.DT_ATVC_SRVC,
    B.DT_DSTV_SRVC,
    A.NR_TAC
FROM TB_TESTE_LINHAS_COMPLETO A
LEFT JOIN P_VIEDB.VW_RLC_LNHA_SRVC B
ON A.ID_LNHA = B.ID_LNHA
    AND B.ID_SRVC IN ('9693','555655','638298','947427','1014701','111946',
    '1036745','1036743','635896','448429','1036741','1967','1014791','118476',
    '1014755','1014719','953522','137962','947425','947429','1036737','558000',
    '1014675','45077','581827','605170','953578','602975','485618','146521',
    '120612','953354','118489','953466','953298','947439','123604','947431',
    '517949','1014773','123649','483073','118475','1036739','111947','981',
    '953186','445826','668060','1014737','134108','579498','953242','1036747',
    '45078','947441','947435','953410','1036751','947433','123605','1014685',
    '520347','137963','1036749','128758','405865','1015235','1015239','1015236',
    '1015241','1015238','1015242','1015237','1015240')
    AND B.DT_ATVC_SRVC <= '{END_DATE}' -- '2023-08-31'
    AND B.DT_DSTV_SRVC > '{END_DATE}' -- '2023-08-31'
--GROUP BY 1,2,3,4,5
QUALIFY ROW_NUMBER() OVER (PARTITION BY A.NR_TLFN ORDER BY B.ID_SRVC DESC) = 1
)
WITH DATA PRIMARY INDEX(ID_LNHA) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_TAC_V1 = r"""
CREATE VOLATILE TABLE TMP_DIM_TAC_V1, NO Log AS(
SELECT
    A.*,
    B.DS_FBRC_ORIG AS FABRICANTE,
    B.DS_MODL_ORIG AS MODELO,
    B.DS_TCNL,
    B.ID_TCNL,
    B.FL_DUAL_CHIP,
    B.FL_SMPH AS FL_SMARTPHONE,
    B.FL_SMPH_LITE AS FL_SMART_LITE,
    B.DS_CRCT_ESPC,
    B.DS_SIST_OPRC,
    B.NM_MODL_GSMA,
    B.DS_SIST_OPRC_GSMA,
    B.DS_CLSF_GSMA,
    B.DS_CLSF_SMPH,
    CASE
        WHEN B.DS_TCNL LIKE '%Band 28%' THEN 1
        WHEN B.DS_TCNL LIKE '%BAND 28%' THEN 1
    ELSE 0 END AS FL_700Mhz
FROM TB_TESTE_LINHAS_COMPLETO A
LEFT JOIN P_VIEDB.VW_DIM_TAC B
ON A.NR_TAC = B.NR_IMEI
--WHERE ID_MES = 202408
QUALIFY Row_Number() Over (PARTITION BY A.NR_TLFN ORDER BY ID_MES DESC) = 1
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_TAC_V2 = r"""
CREATE VOLATILE TABLE TMP_DIM_TAC_V2, NO Log AS(
SELECT
    A.*,
    CASE
        WHEN B.NM_TCNL = 'GSM' THEN '2G'
        WHEN B.NM_TCNL = 'WCDMA' THEN '3G'
        WHEN B.NM_TCNL = 'LTE' THEN '4G'
    ELSE B.NM_TCNL END AS TECNOLOGIA_APARELHO
FROM TMP_DIM_TAC_V1 A
LEFT JOIN P_VIEDB.VW_DIM_TCNL B
ON A.ID_TCNL = B.ID_TCNL
--QUALIFY Row_Number() Over (PARTITION BY A.NR_TAC ORDER BY A.NR_TAC) = 1
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_SRAL_LNHA = r"""
CREATE VOLATILE TABLE TMP_SRAL_LNHA, NO Log AS(
SELECT DISTINCT
    A.*,
    B.ID_TIPO_SRAL,
    B.NR_SRAL
FROM TMP_DIM_TAC_V2 A
LEFT JOIN P_VIEDB.VW_RLC_SRAL_LNHA B
ON A.ID_LNHA = B.ID_LNHA
WHERE (DT_VGNC_INCL <= (DATE - Extract(DAY From DATE)) AND DT_VGNC_FNAL > (DATE - Extract(DAY From DATE)))
QUALIFY Row_Number() Over(PARTITION BY A.NR_TLFN ORDER BY A.NR_TLFN, B.DT_VGNC_INCL DESC ) = 1
)
WITH DATA PRIMARY INDEX(NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_DIM_SRAL_V1 = r"""
CREATE VOLATILE TABLE TMP_DIM_SRAL_V1, NO Log AS(
SELECT
    A.*,
    ID_LNHA_PRMR_ATVC
FROM TMP_SRAL_LNHA A
LEFT JOIN P_VIEDB.VW_DIM_SRAL B
ON A.NR_SRAL = B.NR_SRAL
QUALIFY Row_Number() Over(PARTITION BY A.NR_TLFN ORDER BY DT_PRMR_ATVC_SRAL DESC ) = 1
)
WITH DATA PRIMARY INDEX(NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

TMP_PRQE_FINAL = r"""
CREATE VOLATILE TABLE TMP_PRQE_FINAL, NO Log AS(
SELECT
    A.IMSI,
    A.NR_TLFN,
    A.NR_SRAL,
    A.DS_CRCT_PLNO,
    A.DS_PLNO,
    A.SEGMENTO,
    CAST(OREPLACE(A.FABRICANTE,',',' -') AS VARCHAR(256)) AS FABRICANTE,
    CAST(OREPLACE(A.MODELO,',',' -') AS VARCHAR(256)) AS MODELO,
    A.DS_SIST_OPRC,
    A.DS_CLSF_GSMA,
    A.TECNOLOGIA_APARELHO,
    A.FL_DUAL_CHIP,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('01','02','03','04',
    '05','06','07','08','09','10','11','12','13','15','16','17','18','23',
    '25','26','27','29','30','31','32','37','40') THEN 1 ELSE 0 END AS FL_CHIP_3G,
    CASE 
        WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('14','19','21','28',
        '34','35','36','38','39','41','42','43','44','45','46','47','48',
        '49','50','51','52','53','54','55','56','57','58','59','60','62','66','67','71') THEN 1
        WHEN Substring(A.NR_SRAL From 1 FOR 6) = '895510' AND Substring(A.NR_SRAL From 10 FOR 2) IN ('92','93','94','95','96','97','98','99') THEN 1	
    ELSE 0 END AS FL_CHIP_4G,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('61','63','64','69') THEN 1 ELSE 0 END AS FL_CHIP_5G,
    CASE WHEN Substring(A.NR_SRAL From 10 FOR 2) IN ('52','54','55','62','63','64') THEN 1 ELSE 0 END AS FL_ESIM
FROM TMP_DIM_SRAL_V1 A
)
WITH DATA PRIMARY INDEX (NR_TLFN) ON COMMIT PRESERVE ROWS;
"""

In [ ]:
# Exportar Informações para Teradata
def query_teradata(query, connection, cursor, verbose=True):
    try:
        sRequest = query
        cursor.execute(sRequest, ignoreErrors=3807)
        if verbose:
            # obtain the warnings and errors for transmitting the data to the database -- the acquisition phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + query
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + query
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = (
                "{fn teradata_nativesql}{fn teradata_logon_sequence_number}" + query
            )
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]
    finally:
        pass


def insert_csv_data(table_name, values: int, cursor, file, connection, verbose=True):
    insert_values = ""
    for i in range(values):
        insert_values += "?, "
    query = "INSERT INTO " + table_name + " (" + insert_values[:-2] + ")"
    print(query)

    try:
        connection.autocommit = False
        try:
            sInsert = "{fn teradata_read_csv(" + file + ")}" + query
            cursor.execute(sInsert)

            # obtain the warnings and errors for transmitting the data to the database -- the acquisition phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = (
                "{fn teradata_nativesql}{fn teradata_logon_sequence_number}" + sInsert
            )
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            connection.commit()

            # obtain the warnings and errors for the apply phase
            sRequest = "{fn teradata_nativesql}{fn teradata_get_warnings}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

            sRequest = "{fn teradata_nativesql}{fn teradata_get_errors}" + sInsert
            cursor.execute(sRequest)
            [print(row) for row in cursor.fetchall()]

        finally:
            connection.autocommit = True
    finally:
        if verbose:
            sRequest = f"SELECT TOP 5 * FROM {table_name}"
            print(sRequest)
            cursor.execute(sRequest, ignoreErrors=3807)
            print([row for row in cursor.fetchall()])


def export_full_table_to_csv(table_name, file_path, cursor):
    try:
        sRequest = f"SELECT * FROM {table_name}"
        sFileName = f"{file_path}/dataPy.csv"
        print(f"Exporting table data to file {sFileName}")
        cursor.execute("{fn teradata_write_csv(" + sFileName + ")}" + sRequest)
    finally:
        print("File exporting was successful!")


file = f"{str(BASE_TAP)}/linhas_para_segmentar.csv"
folder = f"{str(BASE_TAP)}"

with teradatasql.connect(
    host=TERADATA_HOST,
    user=TERADATA_USER,
    password=TERADATA_PASSWORD,
    database="P_VIEDB",
    logmech="LDAP",
) as connect:
    with connect.cursor() as cur:
        query_teradata(SET_QUERY_BAND, connect, cur)
        query_teradata(RMNG_OUT_MAI, connect, cur)

        insert_csv_data("RMNG_OUT_MAI", 2, cur, file, connect)

        query_teradata(TB_TESTE_LINHAS_ID_PLNO, connect, cur)
        query_teradata(TB_TESTE_LINHAS_COMPLETO, connect, cur)
        query_teradata(TB_TESTE_LINHAS_SRVC, connect, cur)
        query_teradata(TMP_DIM_TAC_V1, connect, cur)
        query_teradata(TMP_DIM_TAC_V2, connect, cur)
        query_teradata(TMP_SRAL_LNHA, connect, cur)
        query_teradata(TMP_DIM_SRAL_V1, connect, cur)
        query_teradata(TMP_PRQE_FINAL, connect, cur)

        export_full_table_to_csv("TMP_PRQE_FINAL", f"{folder}", cur)

In [ ]:
# Importar informações segmentadas
linhas_segmentadas = pandas.read_csv(
    f"{folder}\\dataPy.csv",
    sep=",",
    encoding="utf_8",
    encoding_errors="ignore",
    on_bad_lines="skip",
    dtype={"IMSI": str, "NR_TLFN": str},
)
linhas_segmentadas.drop_duplicates(subset="IMSI", keep="first", inplace=True)
linhas_segmentadas["IMSI"] = linhas_segmentadas["IMSI"].astype(str)
tap["IMSI"] = tap["IMSI"].astype(str)

print("Cruzando informações...")

for col in [
    "PMN (Settlement) Country",
    "PMN (Settlement) Name",
    "Year-Month (Call)",
    "PMN (Settlement) TADIG Code",
    "Call Type",
]:
    tap[col] = tap[col].astype("category")

for col in [
    "IMSI",
    "MSISDN",
    "Device TAC Code",
    "Number of Calls (incl. combined partials)",
    "Charged SMS",
    "Charged Minutes",
    "Charged MB",
    "Settlement Gross Charge - TAP Local Currency" "Settlement Charge - USD",
    "Settlement Tax - USD",
    "Settlement Gross Charge - USD",
    "Settlement Gross Charge - EUR",
]:
    try:
        tap[col] = pandas.to_numeric(tap[col], downcast="float")
    except KeyError:
        continue
    except ValueError:
        continue

tap["MSISDN_fix"] = tap["MSISDN_fix"].astype(str).str.replace(".0", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].str.replace("<NA>", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].astype("category")

tap["IMSI"] = tap["IMSI"].astype(str).str.replace(".0", "")
tap["IMSI"] = tap["IMSI"].astype(str).str.replace("nan", "")
tap["IMSI"] = tap["IMSI"].str.replace("<NA>", "")
tap["IMSI"] = tap["IMSI"].astype("category")


# Mesclar informações segmentadas
final = tap.merge(linhas_segmentadas, on="IMSI", how="left")

# del tarifas
del linhas_segmentadas
del tap


def add_values_to_dataframe(conditions: list, values: list, default=0) -> pandas.Series:
    return numpy.select(conditions, values, default=default)


print("Ajustando colunas...")

conditions = [
    final["SEGMENTO"].str.startswith("PF_POS") == True,
    final["SEGMENTO"].str.startswith("PF_RESIDENCIAL") == True,
    final["SEGMENTO"].str.startswith("PF_PRE") == True,
    final["SEGMENTO"].str.startswith("PF_CONTROLE") == True,
    final["SEGMENTO"].str.startswith("PJ") == True,
]
values = ["B2C_POS", "B2C_POS", "B2C_PRE", "B2C_CONTROLE", "B2B"]
final["SEGMENTO_fix"] = add_values_to_dataframe(conditions, values, default=pandas.NA)

# Atribuir os tipos corretos e excluir colunas desnecessárias
cols = [
    "PMN (Settlement) Country",
    "PMN (Settlement) TADIG Code",
    "Call Type",
    "IMSI",
    "MSISDN",
    "MSISDN_fix",
    "NR_TLFN",
    "NR_SRAL",
    "DS_CRCT_PLNO",
    "DS_PLNO",
    "SEGMENTO",
    "FABRICANTE",
    "MODELO",
    "DS_SIST_OPRC",
    "DS_CLSF_GSMA",
    "TECNOLOGIA_APARELHO",
    "SEGMENTO_fix",
    "FL_DUAL_CHIP",
    "FL_CHIP_3G",
    "FL_CHIP_4G",
    "FL_CHIP_5G",
    "FL_ESIM",
]
for col in cols:
    try:
        final[col] = final[col].astype("category")
    except KeyError:
        continue

print("Criando colunas para análise de diárias...")
# Para Diárias
final["MOC"] = final[final["Call Type"] == "MOC"]["Charged Minutes"]
final["MOC"] = final["MOC"].astype(str).replace("nan", "0").astype(float)

final["MTC"] = final[final["Call Type"] == "MTC"]["Charged Minutes"]
final["MTC"] = final["MTC"].astype(str).replace("nan", "0").astype(float)

final["Voz"] = final["MOC"] + final["MTC"]

dados_utilizados = (
    final.groupby(by=["SEGMENTO_fix"])
    .agg({"Charged MB": "sum", "IMSI": "nunique"})
    .reset_index()
)
dados_utilizados["Charged TB"] = dados_utilizados["Charged MB"] / 1024**2
print(dados_utilizados)
print(
    "Não Identificado: ",
    final[final["SEGMENTO_fix"].isna()]["IMSI"].nunique(),
    " IMSIs",
)
print(
    "Não Identificado: ",
    final[final["SEGMENTO_fix"].isna()]["Charged MB"].sum() / 1024**2,
    " TB",
)

final.to_parquet(str(BASE_TAP) + f"/{MES_TAP}_NACIONAL.parquet")

In [22]:
for col in [
    "IMSI",
    "MSISDN",
    "Device TAC Code",
    "Number of Calls (incl. combined partials)",
    "Charged SMS",
    "Charged Minutes",
    "Charged MB",
    "Settlement Gross Charge - TAP Local Currency" "Settlement Charge - USD",
    "Settlement Tax - USD",
    "Settlement Gross Charge - USD",
    "Settlement Gross Charge - EUR",
]:
    try:
        tap[col] = pandas.to_numeric(tap[col], downcast="float")
    except KeyError:
        continue
    except ValueError:
        continue

tap["MSISDN_fix"] = tap["MSISDN_fix"].astype(str).str.replace(".0", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].str.replace("<NA>", "")
tap["MSISDN_fix"] = tap["MSISDN_fix"].astype("category")

tap["IMSI"] = tap["IMSI"].astype(str).str.replace(".0", "")
tap["IMSI"] = tap["IMSI"].astype(str).str.replace("nan", "")
tap["IMSI"] = tap["IMSI"].str.replace("<NA>", "")
tap["IMSI"] = tap["IMSI"].astype("category")


# Mesclar informações segmentadas
final = tap.merge(linhas_segmentadas, on="IMSI", how="left")

# del tarifas
del linhas_segmentadas
del tap


def add_values_to_dataframe(conditions: list, values: list, default=0) -> pandas.Series:
    return numpy.select(conditions, values, default=default)


print("Ajustando colunas...")

conditions = [
    final["SEGMENTO"].str.startswith("PF_POS") == True,
    final["SEGMENTO"].str.startswith("PF_RESIDENCIAL") == True,
    final["SEGMENTO"].str.startswith("PF_PRE") == True,
    final["SEGMENTO"].str.startswith("PF_CONTROLE") == True,
    final["SEGMENTO"].str.startswith("PJ") == True,
]
values = ["B2C_POS", "B2C_POS", "B2C_PRE", "B2C_CONTROLE", "B2B"]
final["SEGMENTO_fix"] = add_values_to_dataframe(conditions, values, default=pandas.NA)

# Atribuir os tipos corretos e excluir colunas desnecessárias
cols = [
    "PMN (Settlement) Country",
    "PMN (Settlement) TADIG Code",
    "Call Type",
    "IMSI",
    "MSISDN",
    "MSISDN_fix",
    "NR_TLFN",
    "NR_SRAL",
    "DS_CRCT_PLNO",
    "DS_PLNO",
    "SEGMENTO",
    "FABRICANTE",
    "MODELO",
    "DS_SIST_OPRC",
    "DS_CLSF_GSMA",
    "TECNOLOGIA_APARELHO",
    "SEGMENTO_fix",
    "FL_DUAL_CHIP",
    "FL_CHIP_3G",
    "FL_CHIP_4G",
    "FL_CHIP_5G",
    "FL_ESIM",
]
for col in cols:
    try:
        final[col] = final[col].astype("category")
    except KeyError:
        continue

print("Criando colunas para análise de diárias...")
# Para Diárias
final["MOC"] = final[final["Call Type"] == "MOC"]["Charged Minutes"]
final["MOC"] = final["MOC"].astype(str).replace("nan", "0").astype(float)

final["MTC"] = final[final["Call Type"] == "MTC"]["Charged Minutes"]
final["MTC"] = final["MTC"].astype(str).replace("nan", "0").astype(float)

final["Voz"] = final["MOC"] + final["MTC"]

dados_utilizados = (
    final.groupby(by=["SEGMENTO_fix"])
    .agg({"Charged MB": "sum", "IMSI": "nunique"})
    .reset_index()
)
dados_utilizados["Charged TB"] = dados_utilizados["Charged MB"] / 1024**2
print(dados_utilizados)
print(
    "Não Identificado: ",
    final[final["SEGMENTO_fix"].isna()]["IMSI"].nunique(),
    " IMSIs",
)
print(
    "Não Identificado: ",
    final[final["SEGMENTO_fix"].isna()]["Charged MB"].sum() / 1024**2,
    " TB",
)

final.to_parquet(str(BASE_TAP) + f"/{MES_TAP}_NACIONAL.parquet")

Ajustando colunas...
Criando colunas para análise de diárias...
   SEGMENTO_fix    Charged MB    IMSI  Charged TB
0           B2B  1.418929e+07  303981   13.531958
1  B2C_CONTROLE  1.996360e+07  651243   19.038771
2       B2C_POS  2.560416e+07  455544   24.418028
3       B2C_PRE  7.958393e+06  304292    7.589715
Não Identificado:  34072  IMSIs
Não Identificado:  0.45473859460734656  TB


In [ ]:
cdg = (
    final.groupby(
        by=[
            "PMN (Settlement) TADIG Code",
            "PMN (Settlement) Country",
            "Date (Call) YYYYMMDD",
            "Call Type",
            "IMSI",
            "NR_TLFN",
            "SEGMENTO",
        ],
        observed=True,
    )
    .agg(
        {
            "Charged SMS": "sum",
            "Charged MB": "sum",
            "MOC": "sum",
            "MTC": "sum",
            "Settlement Gross Charge - TAP Local Currency": "sum",
        }
    )
    .reset_index()
)


cdg["CN"] = cdg["NR_TLFN"].astype(str).str[0:2]

cdg["TARIFA_GPRS"] = (
    cdg["Settlement Gross Charge - TAP Local Currency"] / cdg["Charged MB"]
)
cdg["TARIFA_MOC"] = cdg["Settlement Gross Charge - TAP Local Currency"] / cdg["MOC"]
cdg["TARIFA_MTC"] = cdg["Settlement Gross Charge - TAP Local Currency"] / cdg["MTC"]
cdg["TARIFA_SMS"] = (
    cdg["Settlement Gross Charge - TAP Local Currency"] / cdg["Charged SMS"]
)

cdg["MOC (Total)"] = cdg["TARIFA_MOC"] * cdg["MOC"]
cdg["MTC (Total)"] = cdg["TARIFA_MTC"] * cdg["MTC"]
cdg["GPRS (Total)"] = cdg["Charged MB"] * cdg["TARIFA_GPRS"]
cdg["SMS-MO (Total)"] = cdg["Charged SMS"] * cdg["TARIFA_SMS"]
cdg["Total"] = (
    cdg["MOC (Total)"]
    + cdg["MTC (Total)"]
    + cdg["GPRS (Total)"]
    + cdg["SMS-MO (Total)"]
)

cdg_path = "C:\\Users\\a0153041\\Telefonica\\Roaming - General\\DMVNO-C\\22 - Bases\\CDG\\Nacional\\Resumos\\"
cdg = (
    cdg.groupby(by=["CN", "SEGMENTO", "Call Type"], observed=True, dropna=False)
    .agg(
        {
            "Charged MB": "sum",
            "Charged SMS": "sum",
            "MOC": "sum",
            "MTC": "sum",
            "Settlement Gross Charge - TAP Local Currency": "sum",
        }
    )
    .reset_index()
    .pivot(
        columns="Call Type",
        index=["CN", "SEGMENTO"],
        values="Settlement Gross Charge - TAP Local Currency",
    )
    .reset_index()
)
cdg.to_excel(
    f"{cdg_path}{MES_TAP}_Nacional.xlsx",
    index=False,
)

# Tests

In [1]:
import pandas

pandas.read_parquet(
    r"c:\Users\a0153041\Telefonica\Roaming - General\DMVNO-C\22 - Bases\dch\silver\2025_03.parquet"
).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18722253 entries, 0 to 18722252
Data columns (total 38 columns):
 #   Column                                     Dtype         
---  ------                                     -----         
 0   TAP File (Current) Name                    category      
 1   TAP File (Current) Processing Date         datetime64[ns]
 2   Year-Month (Invoicing TAP)                 category      
 3   Date (Call) YYYYMMDD                       datetime64[ns]
 4   PMN (Settlement) Country                   category      
 5   PMN (Settlement) TADIG Code                category      
 6   Call Type                                  category      
 7   IMSI                                       int64         
 8   MSISDN                                     float64       
 9   Device TAC Code                            float64       
 10  Number of Calls (incl. combined partials)  uint16        
 11  Charged SMS                                float32       
 12